# Análise dos dados das variáveis relacionadas ao IDEB

O notebook contém experimentos com essas variáveis. São utilizadas duas bases de dados:<br>
- Váriáveis derivadas do SAEB (informações socioeconômicas)<br> 
- Variáveis com informações das escolas e professores<br>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, RFE, RFECV, SelectFromModel
from sklearn.feature_selection import mutual_info_classif, f_classif, chi2
from sklearn.preprocessing import LabelEncoder
import numpy as np
from numpy import array 
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

## Importando bases de dados

In [ ]:
#Importando variáveis derivadas do SAEB

#base_estudantes_se = pd.read_csv('C:/Users/maril/Documents/ufs/TCC e Pesquisa IDEB/variaveis_derivadas_se.csv')

#base_estudantes_se = pd.read_csv('/content/drive/MyDrive/TCC/variaveis_derivadas_se.csv')

#base_estudantes_se.set_index('ID_ESCOLA', inplace=True)

#base_estudantes_al = pd.read_csv('C:/Users/maril/Documents/ufs/TCC e Pesquisa IDEB/variaveis_derivadas_al.csv')

base_estudantes_al = pd.read_csv('/content/drive/MyDrive/TCC/variaveis_derivadas_al.csv')

#base_estudantes_al.set_index('ID_ESCOLA', inplace=True)

#base_estudantes_ba = pd.read_csv('/content/drive/MyDrive/TCC/variaveis_derivadas_ba.csv')

dados_pessoais = base_estudantes_al

#base_estudantes_ba.set_index('ID_ESCOLA', inplace=True)

#dados_pessoais = pd.merge(base_estudantes_se, base_estudantes_al, how = 'outer')

#dados_pessoais = pd.merge(base_estudantes_ba, dados_pessoais, how = 'outer')

In [ ]:
dados_pessoais.set_index('ID_ESCOLA', inplace=True)

In [ ]:
dados_pessoais

,ID_PROVA_BRASIL,ID_REGIAO,ID_UF,ID_MUNICIPIO,ID_AREA,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,IN_SITUACAO_CENSO,TX_RESP_Q001_A,TX_RESP_Q001_B,TX_RESP_Q020_A,TX_RESP_Q020_B,TX_RESP_Q021_A,TX_RESP_Q021_B,TX_RESP_Q024_A,TX_RESP_Q024_B,TX_RESP_Q025_A,TX_RESP_Q025_B,TX_RESP_Q027_A,TX_RESP_Q027_B,TX_RESP_Q028_A,TX_RESP_Q028_B,TX_RESP_Q029_A,TX_RESP_Q029_B,TX_RESP_Q030_A,TX_RESP_Q030_B,TX_RESP_Q031_A,TX_RESP_Q031_B,TX_RESP_Q038_A,TX_RESP_Q038_B,TX_RESP_Q043_A,TX_RESP_Q043_B,TX_RESP_Q044_A,TX_RESP_Q044_B,TX_RESP_Q052_A,TX_RESP_Q052_B,TX_RESP_Q018_A,TX_RESP_Q018_B,TX_RESP_Q018_C,TX_RESP_Q022_A,...,TX_RESP_Q002_F,TX_RESP_Q016_A,TX_RESP_Q016_B,TX_RESP_Q016_C,TX_RESP_Q016_D,TX_RESP_Q016_E,TX_RESP_Q016_F,TX_RESP_Q019_A,TX_RESP_Q019_B,TX_RESP_Q019_C,TX_RESP_Q019_D,TX_RESP_Q019_E,TX_RESP_Q019_F,TX_RESP_Q019_G,TX_RESP_Q023_A,TX_RESP_Q023_B,TX_RESP_Q023_C,TX_RESP_Q023_D,TX_RESP_Q023_E,TX_RESP_Q023_F,TX_RESP_Q023_G,TX_RESP_Q004_A,TX_RESP_Q004_B,TX_RESP_Q004_C,TX_RESP_Q004_D,TX_RESP_Q004_E,TX_RESP_Q004_F,TX_RESP_Q004_G,TX_RESP_Q004_H,TX_RESP_Q003_A,TX_RESP_Q003_B,TX_RESP_Q003_C,TX_RESP_Q003_D,TX_RESP_Q003_E,TX_RESP_Q003_F,TX_RESP_Q003_G,TX_RESP_Q003_H,TX_RESP_Q003_I,TX_RESP_Q003_J,TX_RESP_Q003_L
ID_ESCOLA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
27000095,2017,2,27,2700102,2,2,1,1,38,59,75,22,68,30,68,28,40,50,92,3,87,8,82,13,93,1,73,22,6,90,27,67,80,16,73,20,86,9,2,68,...,2,0,7,16,21,21,33,10,32,8,6,17,9,16,14,28,14,8,3,2,26,0,1,32,28,11,17,4,5,9,6,6,7,9,11,9,11,7,7,7
61188278,2017,2,27,2700201,2,2,1,1,39,73,92,28,73,45,78,34,54,56,113,4,104,11,100,15,111,3,75,40,16,98,17,97,99,17,70,43,94,17,5,68,...,5,0,15,26,42,18,18,13,32,25,7,26,5,11,15,26,20,9,17,3,24,0,13,41,38,14,6,2,5,12,11,14,13,13,14,11,13,4,3,5
27016331,2017,2,27,2700300,2,2,1,1,45,100,121,26,101,45,120,26,85,59,144,3,131,15,123,22,143,2,94,50,23,121,19,126,127,14,85,59,123,16,8,91,...,4,0,16,30,41,39,21,7,45,29,19,27,12,7,9,46,23,15,31,3,18,0,7,54,52,23,3,3,5,13,13,19,11,15,10,14,13,10,7,11
27016366,2017,2,27,2700300,2,2,1,1,22,18,36,5,32,9,29,13,20,22,39,3,34,8,33,9,36,6,26,16,23,19,16,26,37,4,27,14,26,15,0,16,...,2,1,3,13,8,10,6,2,14,8,5,5,3,4,6,12,5,2,6,1,10,0,0,6,8,8,9,3,7,4,3,4,3,5,5,3,2,1,3,4
27016382,2017,2,27,2700300,2,2,1,1,17,9,23,2,20,5,21,5,16,10,25,1,21,5,21,5,24,2,15,11,5,21,3,23,21,3,14,12,20,5,1,14,...,1,0,3,3,13,5,2,1,8,3,4,6,3,1,1,6,5,6,3,0,5,0,0,6,14,1,2,2,1,3,3,3,4,2,4,1,2,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61206951,2017,2,27,6307176,2,4,1,1,8,13,20,1,18,3,20,1,15,6,21,0,20,1,17,4,21,0,18,3,1,20,2,19,20,1,9,11,20,0,1,15,...,0,0,2,8,5,4,2,0,0,1,2,7,9,2,0,0,2,2,10,3,4,0,5,12,4,0,0,0,0,2,3,2,2,2,2,4,0,0,1,2
61178197,2017,2,27,6307192,2,4,1,1,7,23,30,0,23,7,29,1,24,5,29,1,27,3,24,6,30,0,21,9,0,29,1,29,20,10,8,21,28,2,0,28,...,0,0,2,8,15,5,0,0,0,0,3,13,12,2,0,0,0,5,17,7,1,0,5,18,5,2,0,0,0,1,3,3,2,2,4,3,1,1,4,3
61197558,2017,2,27,6307192,2,4,1,1,6,6,12,0,10,2,11,0,8,3,12,0,11,1,11,1,12,0,9,3,0,12,1,11,7,5,12,0,10,0,2,7,...,1,0,1,2,5,4,0,1,0,0,1,8,1,1,0,0,0,1,9,0,2,0,1,7,3,1,0,0,0,0,1,0,1,2,0,3,0,1,1,2


**Pré-processamento da base de dados pessoais**

In [ ]:
#Função que verifica se o ID de uma escola é inválido
#Segundo o Dicionário de Dados da base o SAEB 6 - Os códigos dos Municípios e 
#das Escolas que começam com o dígito "6" são máscaras, isto é, são códigos fictícios.
def retira_id_ficticio(id_escola):
    if id_escola[0] == "6":
        return True
    else:
        return False

In [ ]:
#Se retirou as escolas fictícias
for escola in dados_pessoais.index:
    if (retira_id_ficticio(str(escola))):
        dados_pessoais.drop(escola, inplace=True)
        

dados_pessoais

,ID_PROVA_BRASIL,ID_REGIAO,ID_UF,ID_MUNICIPIO,ID_AREA,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,IN_SITUACAO_CENSO,TX_RESP_Q001_A,TX_RESP_Q001_B,TX_RESP_Q020_A,TX_RESP_Q020_B,TX_RESP_Q021_A,TX_RESP_Q021_B,TX_RESP_Q024_A,TX_RESP_Q024_B,TX_RESP_Q025_A,TX_RESP_Q025_B,TX_RESP_Q027_A,TX_RESP_Q027_B,TX_RESP_Q028_A,TX_RESP_Q028_B,TX_RESP_Q029_A,TX_RESP_Q029_B,TX_RESP_Q030_A,TX_RESP_Q030_B,TX_RESP_Q031_A,TX_RESP_Q031_B,TX_RESP_Q038_A,TX_RESP_Q038_B,TX_RESP_Q043_A,TX_RESP_Q043_B,TX_RESP_Q044_A,TX_RESP_Q044_B,TX_RESP_Q052_A,TX_RESP_Q052_B,TX_RESP_Q018_A,TX_RESP_Q018_B,TX_RESP_Q018_C,TX_RESP_Q022_A,...,TX_RESP_Q002_F,TX_RESP_Q016_A,TX_RESP_Q016_B,TX_RESP_Q016_C,TX_RESP_Q016_D,TX_RESP_Q016_E,TX_RESP_Q016_F,TX_RESP_Q019_A,TX_RESP_Q019_B,TX_RESP_Q019_C,TX_RESP_Q019_D,TX_RESP_Q019_E,TX_RESP_Q019_F,TX_RESP_Q019_G,TX_RESP_Q023_A,TX_RESP_Q023_B,TX_RESP_Q023_C,TX_RESP_Q023_D,TX_RESP_Q023_E,TX_RESP_Q023_F,TX_RESP_Q023_G,TX_RESP_Q004_A,TX_RESP_Q004_B,TX_RESP_Q004_C,TX_RESP_Q004_D,TX_RESP_Q004_E,TX_RESP_Q004_F,TX_RESP_Q004_G,TX_RESP_Q004_H,TX_RESP_Q003_A,TX_RESP_Q003_B,TX_RESP_Q003_C,TX_RESP_Q003_D,TX_RESP_Q003_E,TX_RESP_Q003_F,TX_RESP_Q003_G,TX_RESP_Q003_H,TX_RESP_Q003_I,TX_RESP_Q003_J,TX_RESP_Q003_L
ID_ESCOLA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
27000095,2017,2,27,2700102,2,2,1,1,38,59,75,22,68,30,68,28,40,50,92,3,87,8,82,13,93,1,73,22,6,90,27,67,80,16,73,20,86,9,2,68,...,2,0,7,16,21,21,33,10,32,8,6,17,9,16,14,28,14,8,3,2,26,0,1,32,28,11,17,4,5,9,6,6,7,9,11,9,11,7,7,7
27016331,2017,2,27,2700300,2,2,1,1,45,100,121,26,101,45,120,26,85,59,144,3,131,15,123,22,143,2,94,50,23,121,19,126,127,14,85,59,123,16,8,91,...,4,0,16,30,41,39,21,7,45,29,19,27,12,7,9,46,23,15,31,3,18,0,7,54,52,23,3,3,5,13,13,19,11,15,10,14,13,10,7,11
27016366,2017,2,27,2700300,2,2,1,1,22,18,36,5,32,9,29,13,20,22,39,3,34,8,33,9,36,6,26,16,23,19,16,26,37,4,27,14,26,15,0,16,...,2,1,3,13,8,10,6,2,14,8,5,5,3,4,6,12,5,2,6,1,10,0,0,6,8,8,9,3,7,4,3,4,3,5,5,3,2,1,3,4
27016382,2017,2,27,2700300,2,2,1,1,17,9,23,2,20,5,21,5,16,10,25,1,21,5,21,5,24,2,15,11,5,21,3,23,21,3,14,12,20,5,1,14,...,1,0,3,3,13,5,2,1,8,3,4,6,3,1,1,6,5,6,3,0,5,0,0,6,14,1,2,2,1,3,3,3,4,2,4,1,2,1,2,0
27016412,2017,2,27,2700300,2,2,1,1,20,26,36,11,29,18,31,16,19,28,47,0,42,5,38,9,46,1,22,25,15,30,5,42,41,6,29,16,42,4,1,29,...,2,0,7,9,11,10,9,2,17,12,3,4,3,6,6,13,10,0,3,2,13,0,0,13,17,6,5,3,3,5,3,2,2,3,5,5,4,7,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27024059,2017,2,27,2709301,2,2,1,1,73,106,134,49,117,66,114,67,84,93,180,3,171,11,152,29,178,4,110,72,27,154,49,128,154,23,127,53,140,29,10,108,...,7,2,16,43,54,32,37,23,47,33,24,24,10,20,32,36,25,19,19,2,49,0,5,48,54,34,14,10,18,13,12,16,14,19,19,17,12,15,19,16
27024067,2017,2,27,2709301,2,2,1,1,21,17,30,9,28,11,24,16,16,23,40,0,39,1,37,3,40,0,29,11,6,32,10,30,31,9,28,12,35,4,0,26,...,3,1,5,8,10,9,6,5,13,4,5,2,2,8,6,12,7,4,2,0,9,1,3,11,12,6,3,3,0,5,1,4,4,1,3,1,4,1,6,5
27025047,2017,2,27,2709301,2,2,1,1,31,59,82,9,65,26,77,15,52,38,89,3,90,2,83,9,90,2,72,20,10,80,5,85,87,3,53,37,77,9,4,59,...,1,0,8,16,40,18,9,5,15,16,15,26,12,2,4,22,18,15,13,3,17,0,5,46,25,10,2,0,3,11,3,8,6,5,7,8,12,7,10,6


In [ ]:
#Importando variáveis com informações das escolas e professores
#base_escola_se = pd.read_csv('C:/Users/maril/Documents/ufs/TCC e Pesquisa IDEB/ESCOLAS2017-SE.csv')

##base_escola_se = pd.read_csv('/content/drive/MyDrive/TCC/ESCOLAS2017-SE.csv')

#base_escola_se.set_index('CO_ENTIDADE', inplace=True)

#base_escola_al = pd.read_csv('C:/Users/maril/Documents/ufs/TCC e Pesquisa IDEB/ESCOLAS2017-AL.csv')

base_escola_al = pd.read_csv('/content/drive/MyDrive/TCC/ESCOLAS2017-AL.csv')

base_escola_al.set_index('CO_ENTIDADE', inplace=True)

#base_escola_ba = pd.read_csv('C:/Users/maril/Documents/ufs/TCC e Pesquisa IDEB/ESCOLAS2017-BA.csv')

#base_escola_ba = pd.read_csv('/content/drive/MyDrive/TCC/ESCOLAS2017-BA.csv')

#base_escola_ba.rename(columns={'Unnamed: 0': 'CO_ENTIDADE'}, inplace=True)

#base_escola_ba.set_index('CO_ENTIDADE', inplace=True)

#dados_escola = pd.merge(base_escola_se, base_escola_al, how = 'outer')

#dados_escola = pd.concat([dados_escola, base_escola_ba], join='outer')

dados_escola = base_escola_al

In [ ]:
#dados_escola.set_index('CO_ENTIDADE', inplace=True)

dados_escola

,NU_ANO_CENSO,NO_ENTIDADE,CO_ORGAO_REGIONAL,TP_SITUACAO_FUNCIONAMENTO,DT_ANO_LETIVO_INICIO,DT_ANO_LETIVO_TERMINO,CO_REGIAO,CO_MESORREGIAO,CO_MICRORREGIAO,CO_UF,CO_MUNICIPIO,CO_DISTRITO,TP_DEPENDENCIA,TP_LOCALIZACAO,TP_CATEGORIA_ESCOLA_PRIVADA,IN_CONVENIADA_PP,TP_CONVENIO_PODER_PUBLICO,IN_MANT_ESCOLA_PRIVADA_EMP,IN_MANT_ESCOLA_PRIVADA_ONG,IN_MANT_ESCOLA_PRIVADA_SIND,IN_MANT_ESCOLA_PRIVADA_SIST_S,IN_MANT_ESCOLA_PRIVADA_S_FINS,CO_ESCOLA_SEDE_VINCULADA,CO_IES_OFERTANTE,TP_REGULAMENTACAO,IN_LOCAL_FUNC_PREDIO_ESCOLAR,TP_OCUPACAO_PREDIO_ESCOLAR,IN_LOCAL_FUNC_SALAS_EMPRESA,IN_LOCAL_FUNC_SOCIOEDUCATIVO,IN_LOCAL_FUNC_UNID_PRISIONAL,IN_LOCAL_FUNC_PRISIONAL_SOCIO,IN_LOCAL_FUNC_TEMPLO_IGREJA,IN_LOCAL_FUNC_CASA_PROFESSOR,IN_LOCAL_FUNC_GALPAO,TP_OCUPACAO_GALPAO,IN_LOCAL_FUNC_SALAS_OUTRA_ESC,IN_LOCAL_FUNC_OUTROS,IN_PREDIO_COMPARTILHADO,IN_AGUA_FILTRADA,IN_AGUA_REDE_PUBLICA,...,IN_COMUM_PRE,IN_COMUM_FUND_AI,IN_COMUM_FUND_AF,IN_COMUM_MEDIO_MEDIO,IN_COMUM_MEDIO_INTEGRADO,IN_COMUM_MEDIO_NORMAL,IN_ESP_EXCLUSIVA_CRECHE,IN_ESP_EXCLUSIVA_PRE,IN_ESP_EXCLUSIVA_FUND_AI,IN_ESP_EXCLUSIVA_FUND_AF,IN_ESP_EXCLUSIVA_MEDIO_MEDIO,IN_ESP_EXCLUSIVA_MEDIO_INTEGR,IN_ESP_EXCLUSIVA_MEDIO_NORMAL,IN_COMUM_EJA_FUND,IN_COMUM_EJA_MEDIO,IN_COMUM_EJA_PROF,IN_ESP_EXCLUSIVA_EJA_FUND,IN_ESP_EXCLUSIVA_EJA_MEDIO,IN_ESP_EXCLUSIVA_EJA_PROF,IN_COMUM_PROF,IN_ESP_EXCLUSIVA_PROF,QtdeDocentesEsp,QtdeDocentesMest,QtdeDocentesDout,QtdeDocentesSemPos,QtdeDocentesFCEM,QtdeDocentesDivSocCulta,QtdeDocentesAtendEsp,QtdeDocentesTotal,QtdeTurmasProf,QtdeDocentes,QtdeDocentesAuxAssist,QtdeDocentesMonit,asQtdeDocentesEfetivos,QtdeDocentesTempo,QtdeDocentesTerceir,QtdedocentesCLT,QtdeDocentesComLicenciatura,QtdeAlunosTotal,Ideb-2017
CO_ENTIDADE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
27000095,2017,ESCOLA ESTADUAL MONSENHOR SEBASTIAO ALVES BEZERRA,11.0,1,10/05/2017,15/12/2017,2,2701,27001,27,2700102,270010205,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,62,0,0,103,0,0,0,172,172,172.0,NaN,NaN,24.0,148.0,NaN,NaN,172.0,508,3.4
27001237,2017,ESCOLA ESTADUAL LUIZ BASTOS,11.0,1,06/02/2017,15/12/2017,2,2701,27001,27,2701605,270160505,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,111,0,0,89,31,0,0,250,250,250.0,NaN,NaN,11.0,225.0,14.0,NaN,250.0,792,3.9
27002020,2017,ESCOLA ESTADUAL GENTIL DE ALBUQUERQUE MALTA,11.0,1,06/03/2017,15/12/2017,2,2701,27001,27,2705002,270500205,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,100,0,0,86,0,0,0,222,222,222.0,NaN,NaN,5.0,217.0,NaN,NaN,222.0,652,3.3
27002063,2017,ESCOLA ESTADUAL SANTA CRUZ DO DESERTO,11.0,1,06/03/2017,15/12/2017,2,2701,27001,27,2705002,270500205,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20,0,0,43,0,0,0,73,73,73.0,NaN,NaN,13.0,40.0,20.0,NaN,63.0,309,3.8
27002950,2017,ESCOLA ESTADUAL DE EDUCACAO BASICA DE PARICONHA,11.0,1,10/04/2017,15/12/2017,2,2701,27001,27,2706422,270642205,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,72,7,0,36,0,0,0,121,121,121.0,NaN,NaN,37.0,84.0,NaN,NaN,117.0,442,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [ ]:
col_ideb = dados_escola['Ideb-2017']

dados_escola.drop('Ideb-2017', axis=1, inplace=True)

In [ ]:
col_ideb

CO_ENTIDADE
27000095    3.4
27001237    3.9
27002020    3.3
27002063    3.8
27002950    3.5
           ... 
27226697    4.3
27226808    3.9
27227189    5.7
27235009    5.1
27353036    3.0
Name: Ideb-2017, Length: 143, dtype: float64

In [ ]:
dados_escola

,NU_ANO_CENSO,NO_ENTIDADE,CO_ORGAO_REGIONAL,TP_SITUACAO_FUNCIONAMENTO,DT_ANO_LETIVO_INICIO,DT_ANO_LETIVO_TERMINO,CO_REGIAO,CO_MESORREGIAO,CO_MICRORREGIAO,CO_UF,CO_MUNICIPIO,CO_DISTRITO,TP_DEPENDENCIA,TP_LOCALIZACAO,TP_CATEGORIA_ESCOLA_PRIVADA,IN_CONVENIADA_PP,TP_CONVENIO_PODER_PUBLICO,IN_MANT_ESCOLA_PRIVADA_EMP,IN_MANT_ESCOLA_PRIVADA_ONG,IN_MANT_ESCOLA_PRIVADA_SIND,IN_MANT_ESCOLA_PRIVADA_SIST_S,IN_MANT_ESCOLA_PRIVADA_S_FINS,CO_ESCOLA_SEDE_VINCULADA,CO_IES_OFERTANTE,TP_REGULAMENTACAO,IN_LOCAL_FUNC_PREDIO_ESCOLAR,TP_OCUPACAO_PREDIO_ESCOLAR,IN_LOCAL_FUNC_SALAS_EMPRESA,IN_LOCAL_FUNC_SOCIOEDUCATIVO,IN_LOCAL_FUNC_UNID_PRISIONAL,IN_LOCAL_FUNC_PRISIONAL_SOCIO,IN_LOCAL_FUNC_TEMPLO_IGREJA,IN_LOCAL_FUNC_CASA_PROFESSOR,IN_LOCAL_FUNC_GALPAO,TP_OCUPACAO_GALPAO,IN_LOCAL_FUNC_SALAS_OUTRA_ESC,IN_LOCAL_FUNC_OUTROS,IN_PREDIO_COMPARTILHADO,IN_AGUA_FILTRADA,IN_AGUA_REDE_PUBLICA,...,IN_COMUM_CRECHE,IN_COMUM_PRE,IN_COMUM_FUND_AI,IN_COMUM_FUND_AF,IN_COMUM_MEDIO_MEDIO,IN_COMUM_MEDIO_INTEGRADO,IN_COMUM_MEDIO_NORMAL,IN_ESP_EXCLUSIVA_CRECHE,IN_ESP_EXCLUSIVA_PRE,IN_ESP_EXCLUSIVA_FUND_AI,IN_ESP_EXCLUSIVA_FUND_AF,IN_ESP_EXCLUSIVA_MEDIO_MEDIO,IN_ESP_EXCLUSIVA_MEDIO_INTEGR,IN_ESP_EXCLUSIVA_MEDIO_NORMAL,IN_COMUM_EJA_FUND,IN_COMUM_EJA_MEDIO,IN_COMUM_EJA_PROF,IN_ESP_EXCLUSIVA_EJA_FUND,IN_ESP_EXCLUSIVA_EJA_MEDIO,IN_ESP_EXCLUSIVA_EJA_PROF,IN_COMUM_PROF,IN_ESP_EXCLUSIVA_PROF,QtdeDocentesEsp,QtdeDocentesMest,QtdeDocentesDout,QtdeDocentesSemPos,QtdeDocentesFCEM,QtdeDocentesDivSocCulta,QtdeDocentesAtendEsp,QtdeDocentesTotal,QtdeTurmasProf,QtdeDocentes,QtdeDocentesAuxAssist,QtdeDocentesMonit,asQtdeDocentesEfetivos,QtdeDocentesTempo,QtdeDocentesTerceir,QtdedocentesCLT,QtdeDocentesComLicenciatura,QtdeAlunosTotal
CO_ENTIDADE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
27000095,2017,ESCOLA ESTADUAL MONSENHOR SEBASTIAO ALVES BEZERRA,11.0,1,10/05/2017,15/12/2017,2,2701,27001,27,2700102,270010205,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,62,0,0,103,0,0,0,172,172,172.0,NaN,NaN,24.0,148.0,NaN,NaN,172.0,508
27001237,2017,ESCOLA ESTADUAL LUIZ BASTOS,11.0,1,06/02/2017,15/12/2017,2,2701,27001,27,2701605,270160505,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,111,0,0,89,31,0,0,250,250,250.0,NaN,NaN,11.0,225.0,14.0,NaN,250.0,792
27002020,2017,ESCOLA ESTADUAL GENTIL DE ALBUQUERQUE MALTA,11.0,1,06/03/2017,15/12/2017,2,2701,27001,27,2705002,270500205,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,100,0,0,86,0,0,0,222,222,222.0,NaN,NaN,5.0,217.0,NaN,NaN,222.0,652
27002063,2017,ESCOLA ESTADUAL SANTA CRUZ DO DESERTO,11.0,1,06/03/2017,15/12/2017,2,2701,27001,27,2705002,270500205,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20,0,0,43,0,0,0,73,73,73.0,NaN,NaN,13.0,40.0,20.0,NaN,63.0,309
27002950,2017,ESCOLA ESTADUAL DE EDUCACAO BASICA DE PARICONHA,11.0,1,10/04/2017,15/12/2017,2,2701,27001,27,2706422,270642205,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,72,7,0,36,0,0,0,121,121,121.0,NaN,NaN,37.0,84.0,NaN,NaN,117.0,442
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

#### Pré-processamento da base das escolas

In [ ]:
#Retirando valores ausentes
filteredColumns = dados_escola.dtypes[dados_escola.dtypes == np.float64]

listOfColumnNames = list(filteredColumns.index)

# retorna o número de valores ausentes para cada coluna
dados_escola[listOfColumnNames].isnull().sum()

CO_ORGAO_REGIONAL                3
TP_CATEGORIA_ESCOLA_PRIVADA    118
IN_CONVENIADA_PP               118
TP_CONVENIO_PODER_PUBLICO      138
IN_MANT_ESCOLA_PRIVADA_EMP     118
                              ... 
asQtdeDocentesEfetivos          29
QtdeDocentesTempo               25
QtdeDocentesTerceir            135
QtdedocentesCLT                143
QtdeDocentesComLicenciatura      0
Length: 160, dtype: int64

In [ ]:
#Retirando colunas que apresentam mais de 60% do valores nulos
dados_escola.dropna(thresh=len(dados_escola)*0.6, axis=1, inplace=True)

#Preenchendo valores ausentes
dados_escola.fillna(0, inplace=True)

filteredColumns = dados_escola.dtypes[dados_escola.dtypes == np.float64]

listOfColumnNames = list(filteredColumns.index)

# retorna o número de valores ausentes para cada coluna
dados_escola[listOfColumnNames].isnull().sum()

CO_ORGAO_REGIONAL               0
TP_REGULAMENTACAO               0
IN_LOCAL_FUNC_PREDIO_ESCOLAR    0
TP_OCUPACAO_PREDIO_ESCOLAR      0
IN_LOCAL_FUNC_SALAS_EMPRESA     0
                               ..
IN_ESP_EXCLUSIVA_PROF           0
QtdeDocentes                    0
asQtdeDocentesEfetivos          0
QtdeDocentesTempo               0
QtdeDocentesComLicenciatura     0
Length: 143, dtype: int64

In [ ]:
dados_escola[listOfColumnNames] = dados_escola[listOfColumnNames].astype(int)

In [ ]:
dados_escola.insert(len(dados_escola.columns), 'Ideb-2017', col_ideb)
#['Ideb-2017'] = col_ideb

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2828: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if self.run_code(code, result):


In [ ]:
dados_escola

,NU_ANO_CENSO,NO_ENTIDADE,CO_ORGAO_REGIONAL,TP_SITUACAO_FUNCIONAMENTO,DT_ANO_LETIVO_INICIO,DT_ANO_LETIVO_TERMINO,CO_REGIAO,CO_MESORREGIAO,CO_MICRORREGIAO,CO_UF,CO_MUNICIPIO,CO_DISTRITO,TP_DEPENDENCIA,TP_LOCALIZACAO,TP_REGULAMENTACAO,IN_LOCAL_FUNC_PREDIO_ESCOLAR,TP_OCUPACAO_PREDIO_ESCOLAR,IN_LOCAL_FUNC_SALAS_EMPRESA,IN_LOCAL_FUNC_SOCIOEDUCATIVO,IN_LOCAL_FUNC_UNID_PRISIONAL,IN_LOCAL_FUNC_PRISIONAL_SOCIO,IN_LOCAL_FUNC_TEMPLO_IGREJA,IN_LOCAL_FUNC_CASA_PROFESSOR,IN_LOCAL_FUNC_GALPAO,IN_LOCAL_FUNC_SALAS_OUTRA_ESC,IN_LOCAL_FUNC_OUTROS,IN_PREDIO_COMPARTILHADO,IN_AGUA_FILTRADA,IN_AGUA_REDE_PUBLICA,IN_AGUA_POCO_ARTESIANO,IN_AGUA_CACIMBA,IN_AGUA_FONTE_RIO,IN_AGUA_INEXISTENTE,IN_ENERGIA_REDE_PUBLICA,IN_ENERGIA_GERADOR,IN_ENERGIA_OUTROS,IN_ENERGIA_INEXISTENTE,IN_ESGOTO_REDE_PUBLICA,IN_ESGOTO_FOSSA,IN_ESGOTO_INEXISTENTE,...,IN_REGULAR,IN_EJA,IN_PROFISSIONALIZANTE,IN_COMUM_CRECHE,IN_COMUM_PRE,IN_COMUM_FUND_AI,IN_COMUM_FUND_AF,IN_COMUM_MEDIO_MEDIO,IN_COMUM_MEDIO_INTEGRADO,IN_COMUM_MEDIO_NORMAL,IN_ESP_EXCLUSIVA_CRECHE,IN_ESP_EXCLUSIVA_PRE,IN_ESP_EXCLUSIVA_FUND_AI,IN_ESP_EXCLUSIVA_FUND_AF,IN_ESP_EXCLUSIVA_MEDIO_MEDIO,IN_ESP_EXCLUSIVA_MEDIO_INTEGR,IN_ESP_EXCLUSIVA_MEDIO_NORMAL,IN_COMUM_EJA_FUND,IN_COMUM_EJA_MEDIO,IN_COMUM_EJA_PROF,IN_ESP_EXCLUSIVA_EJA_FUND,IN_ESP_EXCLUSIVA_EJA_MEDIO,IN_ESP_EXCLUSIVA_EJA_PROF,IN_COMUM_PROF,IN_ESP_EXCLUSIVA_PROF,QtdeDocentesEsp,QtdeDocentesMest,QtdeDocentesDout,QtdeDocentesSemPos,QtdeDocentesFCEM,QtdeDocentesDivSocCulta,QtdeDocentesAtendEsp,QtdeDocentesTotal,QtdeTurmasProf,QtdeDocentes,asQtdeDocentesEfetivos,QtdeDocentesTempo,QtdeDocentesComLicenciatura,QtdeAlunosTotal,Ideb-2017
CO_ENTIDADE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
27000095,2017,ESCOLA ESTADUAL MONSENHOR SEBASTIAO ALVES BEZERRA,11,1,10/05/2017,15/12/2017,2,2701,27001,27,2700102,270010205,2,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,...,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,62,0,0,103,0,0,0,172,172,172,24,148,172,508,3.4
27001237,2017,ESCOLA ESTADUAL LUIZ BASTOS,11,1,06/02/2017,15/12/2017,2,2701,27001,27,2701605,270160505,2,1,1,1,1,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,0,1,0,0,0,0,1,0,...,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,111,0,0,89,31,0,0,250,250,250,11,225,250,792,3.9
27002020,2017,ESCOLA ESTADUAL GENTIL DE ALBUQUERQUE MALTA,11,1,06/03/2017,15/12/2017,2,2701,27001,27,2705002,270500205,2,1,2,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,...,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,100,0,0,86,0,0,0,222,222,222,5,217,222,652,3.3
27002063,2017,ESCOLA ESTADUAL SANTA CRUZ DO DESERTO,11,1,06/03/2017,15/12/2017,2,2701,27001,27,2705002,270500205,2,2,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,...,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,20,0,0,43,0,0,0,73,73,73,13,40,63,309,3.8
27002950,2017,ESCOLA ESTADUAL DE EDUCACAO BASICA DE PARICONHA,11,1,10/04/2017,15/12/2017,2,2701,27001,27,2706422,270642205,2,1,2,1,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,1,0,...,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,72,7,0,36,0,0,0,121,121,121,37,84,117,442,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27226697,2017,COLEGIO ROSALVO FELIX,12,1,30/01/2017,22/12/2017,2,2703,27011,27,2707701,270770105,4,1,2,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,...,1,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,51,0,0,0,148,148,146,0,0,105,480,4.3
27226808,2017,ESCOLA ESTADUAL PROFA ELZA SOARES CAVALCANTE,3,1,08/02/2017,14/12/2017,2,2702,27005,27,2707602,270760205,2,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,...,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,33,0,0,92,6,0,0,168,168,168,79,89,125,397,3.9
27227189,2017,COLEGIO DE EDUCACAO BASICA SANTA ESMERALDA,5,1,16/01/201

In [ ]:
#Colunas categóricas
filteredColumns = dados_escola.dtypes[dados_escola.dtypes == np.object]

listOfColumnNames = list(filteredColumns.index)

dados_escola[listOfColumnNames]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


,NO_ENTIDADE,DT_ANO_LETIVO_INICIO,DT_ANO_LETIVO_TERMINO
CO_ENTIDADE,,,
27000095,ESCOLA ESTADUAL MONSENHOR SEBASTIAO ALVES BEZERRA,10/05/2017,15/12/2017
27001237,ESCOLA ESTADUAL LUIZ BASTOS,06/02/2017,15/12/2017
27002020,ESCOLA ESTADUAL GENTIL DE ALBUQUERQUE MALTA,06/03/2017,15/12/2017
27002063,ESCOLA ESTADUAL SANTA CRUZ DO DESERTO,06/03/2017,15/12/2017
27002950,ESCOLA ESTADUAL DE EDUCACAO BASICA DE PARICONHA,10/04/2017,15/12/2017
...,...,...,...
27226697,COLEGIO ROSALVO FELIX,30/01/2017,22/12/2017
27226808,ESCOLA ESTADUAL PROFA ELZA SOARES CAVALCANTE,08/02/2017,14/12/2017
27227189,COLEGIO DE EDUCACAO BASICA SANTA ESMERALDA,16/01/2017,16/12/2017


In [ ]:
#Tratando datas, convertendo para timestamp

inicio = pd.to_datetime(dados_escola['DT_ANO_LETIVO_INICIO'], infer_datetime_format = True)
termino = pd.to_datetime(dados_escola['DT_ANO_LETIVO_TERMINO'], infer_datetime_format = True)

dados_escola['DT_ANO_LETIVO_INICIO'] = inicio.values.astype (np.int64)
dados_escola['DT_ANO_LETIVO_TERMINO'] = termino.values.astype (np.int64)

In [ ]:
dados_escola.drop('NO_ENTIDADE', axis=1, inplace=True)

In [ ]:
dados_escola.columns.to_list()

['NU_ANO_CENSO',
 'CO_ORGAO_REGIONAL',
 'TP_SITUACAO_FUNCIONAMENTO',
 'DT_ANO_LETIVO_INICIO',
 'DT_ANO_LETIVO_TERMINO',
 'CO_REGIAO',
 'CO_MESORREGIAO',
 'CO_MICRORREGIAO',
 'CO_UF',
 'CO_MUNICIPIO',
 'CO_DISTRITO',
 'TP_DEPENDENCIA',
 'TP_LOCALIZACAO',
 'TP_REGULAMENTACAO',
 'IN_LOCAL_FUNC_PREDIO_ESCOLAR',
 'TP_OCUPACAO_PREDIO_ESCOLAR',
 'IN_LOCAL_FUNC_SALAS_EMPRESA',
 'IN_LOCAL_FUNC_SOCIOEDUCATIVO',
 'IN_LOCAL_FUNC_UNID_PRISIONAL',
 'IN_LOCAL_FUNC_PRISIONAL_SOCIO',
 'IN_LOCAL_FUNC_TEMPLO_IGREJA',
 'IN_LOCAL_FUNC_CASA_PROFESSOR',
 'IN_LOCAL_FUNC_GALPAO',
 'IN_LOCAL_FUNC_SALAS_OUTRA_ESC',
 'IN_LOCAL_FUNC_OUTROS',
 'IN_PREDIO_COMPARTILHADO',
 'IN_AGUA_FILTRADA',
 'IN_AGUA_REDE_PUBLICA',
 'IN_AGUA_POCO_ARTESIANO',
 'IN_AGUA_CACIMBA',
 'IN_AGUA_FONTE_RIO',
 'IN_AGUA_INEXISTENTE',
 'IN_ENERGIA_REDE_PUBLICA',
 'IN_ENERGIA_GERADOR',
 'IN_ENERGIA_OUTROS',
 'IN_ENERGIA_INEXISTENTE',
 'IN_ESGOTO_REDE_PUBLICA',
 'IN_ESGOTO_FOSSA',
 'IN_ESGOTO_INEXISTENTE',
 'IN_LIXO_COLETA_PERIODICA',
 'IN_LIXO_

In [ ]:
dados_pessoais

,ID_PROVA_BRASIL,ID_REGIAO,ID_UF,ID_MUNICIPIO,ID_AREA,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,IN_SITUACAO_CENSO,TX_RESP_Q001_A,TX_RESP_Q001_B,TX_RESP_Q020_A,TX_RESP_Q020_B,TX_RESP_Q021_A,TX_RESP_Q021_B,TX_RESP_Q024_A,TX_RESP_Q024_B,TX_RESP_Q025_A,TX_RESP_Q025_B,TX_RESP_Q027_A,TX_RESP_Q027_B,TX_RESP_Q028_A,TX_RESP_Q028_B,TX_RESP_Q029_A,TX_RESP_Q029_B,TX_RESP_Q030_A,TX_RESP_Q030_B,TX_RESP_Q031_A,TX_RESP_Q031_B,TX_RESP_Q038_A,TX_RESP_Q038_B,TX_RESP_Q043_A,TX_RESP_Q043_B,TX_RESP_Q044_A,TX_RESP_Q044_B,TX_RESP_Q052_A,TX_RESP_Q052_B,TX_RESP_Q018_A,TX_RESP_Q018_B,TX_RESP_Q018_C,TX_RESP_Q022_A,...,TX_RESP_Q002_F,TX_RESP_Q016_A,TX_RESP_Q016_B,TX_RESP_Q016_C,TX_RESP_Q016_D,TX_RESP_Q016_E,TX_RESP_Q016_F,TX_RESP_Q019_A,TX_RESP_Q019_B,TX_RESP_Q019_C,TX_RESP_Q019_D,TX_RESP_Q019_E,TX_RESP_Q019_F,TX_RESP_Q019_G,TX_RESP_Q023_A,TX_RESP_Q023_B,TX_RESP_Q023_C,TX_RESP_Q023_D,TX_RESP_Q023_E,TX_RESP_Q023_F,TX_RESP_Q023_G,TX_RESP_Q004_A,TX_RESP_Q004_B,TX_RESP_Q004_C,TX_RESP_Q004_D,TX_RESP_Q004_E,TX_RESP_Q004_F,TX_RESP_Q004_G,TX_RESP_Q004_H,TX_RESP_Q003_A,TX_RESP_Q003_B,TX_RESP_Q003_C,TX_RESP_Q003_D,TX_RESP_Q003_E,TX_RESP_Q003_F,TX_RESP_Q003_G,TX_RESP_Q003_H,TX_RESP_Q003_I,TX_RESP_Q003_J,TX_RESP_Q003_L
ID_ESCOLA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
27000095,2017,2,27,2700102,2,2,1,1,38,59,75,22,68,30,68,28,40,50,92,3,87,8,82,13,93,1,73,22,6,90,27,67,80,16,73,20,86,9,2,68,...,2,0,7,16,21,21,33,10,32,8,6,17,9,16,14,28,14,8,3,2,26,0,1,32,28,11,17,4,5,9,6,6,7,9,11,9,11,7,7,7
27016331,2017,2,27,2700300,2,2,1,1,45,100,121,26,101,45,120,26,85,59,144,3,131,15,123,22,143,2,94,50,23,121,19,126,127,14,85,59,123,16,8,91,...,4,0,16,30,41,39,21,7,45,29,19,27,12,7,9,46,23,15,31,3,18,0,7,54,52,23,3,3,5,13,13,19,11,15,10,14,13,10,7,11
27016366,2017,2,27,2700300,2,2,1,1,22,18,36,5,32,9,29,13,20,22,39,3,34,8,33,9,36,6,26,16,23,19,16,26,37,4,27,14,26,15,0,16,...,2,1,3,13,8,10,6,2,14,8,5,5,3,4,6,12,5,2,6,1,10,0,0,6,8,8,9,3,7,4,3,4,3,5,5,3,2,1,3,4
27016382,2017,2,27,2700300,2,2,1,1,17,9,23,2,20,5,21,5,16,10,25,1,21,5,21,5,24,2,15,11,5,21,3,23,21,3,14,12,20,5,1,14,...,1,0,3,3,13,5,2,1,8,3,4,6,3,1,1,6,5,6,3,0,5,0,0,6,14,1,2,2,1,3,3,3,4,2,4,1,2,1,2,0
27016412,2017,2,27,2700300,2,2,1,1,20,26,36,11,29,18,31,16,19,28,47,0,42,5,38,9,46,1,22,25,15,30,5,42,41,6,29,16,42,4,1,29,...,2,0,7,9,11,10,9,2,17,12,3,4,3,6,6,13,10,0,3,2,13,0,0,13,17,6,5,3,3,5,3,2,2,3,5,5,4,7,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27024059,2017,2,27,2709301,2,2,1,1,73,106,134,49,117,66,114,67,84,93,180,3,171,11,152,29,178,4,110,72,27,154,49,128,154,23,127,53,140,29,10,108,...,7,2,16,43,54,32,37,23,47,33,24,24,10,20,32,36,25,19,19,2,49,0,5,48,54,34,14,10,18,13,12,16,14,19,19,17,12,15,19,16
27024067,2017,2,27,2709301,2,2,1,1,21,17,30,9,28,11,24,16,16,23,40,0,39,1,37,3,40,0,29,11,6,32,10,30,31,9,28,12,35,4,0,26,...,3,1,5,8,10,9,6,5,13,4,5,2,2,8,6,12,7,4,2,0,9,1,3,11,12,6,3,3,0,5,1,4,4,1,3,1,4,1,6,5
27025047,2017,2,27,2709301,2,2,1,1,31,59,82,9,65,26,77,15,52,38,89,3,90,2,83,9,90,2,72,20,10,80,5,85,87,3,53,37,77,9,4,59,...,1,0,8,16,40,18,9,5,15,16,15,26,12,2,4,22,18,15,13,3,17,0,5,46,25,10,2,0,3,11,3,8,6,5,7,8,12,7,10,6


### Unindo as bases de dados 

Foi realizado join nos dois datasets, nesse momento se percebeu que não existem variáveis escolas particulares na base derivada do SAEB

In [ ]:
dataset = dados_pessoais.join(dados_escola, how="inner")

In [ ]:
dataset

,ID_PROVA_BRASIL,ID_REGIAO,ID_UF,ID_MUNICIPIO,ID_AREA,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,IN_SITUACAO_CENSO,TX_RESP_Q001_A,TX_RESP_Q001_B,TX_RESP_Q020_A,TX_RESP_Q020_B,TX_RESP_Q021_A,TX_RESP_Q021_B,TX_RESP_Q024_A,TX_RESP_Q024_B,TX_RESP_Q025_A,TX_RESP_Q025_B,TX_RESP_Q027_A,TX_RESP_Q027_B,TX_RESP_Q028_A,TX_RESP_Q028_B,TX_RESP_Q029_A,TX_RESP_Q029_B,TX_RESP_Q030_A,TX_RESP_Q030_B,TX_RESP_Q031_A,TX_RESP_Q031_B,TX_RESP_Q038_A,TX_RESP_Q038_B,TX_RESP_Q043_A,TX_RESP_Q043_B,TX_RESP_Q044_A,TX_RESP_Q044_B,TX_RESP_Q052_A,TX_RESP_Q052_B,TX_RESP_Q018_A,TX_RESP_Q018_B,TX_RESP_Q018_C,TX_RESP_Q022_A,...,IN_REGULAR,IN_EJA,IN_PROFISSIONALIZANTE,IN_COMUM_CRECHE,IN_COMUM_PRE,IN_COMUM_FUND_AI,IN_COMUM_FUND_AF,IN_COMUM_MEDIO_MEDIO,IN_COMUM_MEDIO_INTEGRADO,IN_COMUM_MEDIO_NORMAL,IN_ESP_EXCLUSIVA_CRECHE,IN_ESP_EXCLUSIVA_PRE,IN_ESP_EXCLUSIVA_FUND_AI,IN_ESP_EXCLUSIVA_FUND_AF,IN_ESP_EXCLUSIVA_MEDIO_MEDIO,IN_ESP_EXCLUSIVA_MEDIO_INTEGR,IN_ESP_EXCLUSIVA_MEDIO_NORMAL,IN_COMUM_EJA_FUND,IN_COMUM_EJA_MEDIO,IN_COMUM_EJA_PROF,IN_ESP_EXCLUSIVA_EJA_FUND,IN_ESP_EXCLUSIVA_EJA_MEDIO,IN_ESP_EXCLUSIVA_EJA_PROF,IN_COMUM_PROF,IN_ESP_EXCLUSIVA_PROF,QtdeDocentesEsp,QtdeDocentesMest,QtdeDocentesDout,QtdeDocentesSemPos,QtdeDocentesFCEM,QtdeDocentesDivSocCulta,QtdeDocentesAtendEsp,QtdeDocentesTotal,QtdeTurmasProf,QtdeDocentes,asQtdeDocentesEfetivos,QtdeDocentesTempo,QtdeDocentesComLicenciatura,QtdeAlunosTotal,Ideb-2017
27000095,2017,2,27,2700102,2,2,1,1,38,59,75,22,68,30,68,28,40,50,92,3,87,8,82,13,93,1,73,22,6,90,27,67,80,16,73,20,86,9,2,68,...,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,62,0,0,103,0,0,0,172,172,172,24,148,172,508,3.4
27001237,2017,2,27,2701605,2,2,1,1,76,108,144,44,130,58,113,68,77,100,177,6,167,16,162,21,179,4,133,49,43,139,55,119,146,36,102,75,153,30,4,123,...,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,111,0,0,89,31,0,0,250,250,250,11,225,250,792,3.9
27002020,2017,2,27,2705002,2,2,1,1,41,60,78,21,70,28,66,29,49,47,95,2,88,9,89,8,91,5,73,24,13,80,36,52,84,8,61,24,79,20,1,64,...,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,100,0,0,86,0,0,0,222,222,222,5,217,222,652,3.3
27002063,2017,2,27,2705002,2,2,2,1,12,24,22,13,15,20,18,18,9,26,34,2,32,4,33,3,35,1,26,9,2,32,14,21,28,7,29,5,33,2,0,26,...,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,20,0,0,43,0,0,0,73,73,73,13,40,63,309,3.8
27002950,2017,2,27,2706422,2,2,1,1,70,72,109,31,88,52,89,46,57,77,134,3,123,13,115,22,133,4,98,38,14,122,36,102,93,46,83,52,116,22,3,81,...,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,72,7,0,36,0,0,0,121,121,121,37,84,117,442,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27226131,2017,2,27,2700706,2,2,1,1,64,86,108,46,97,57,96,57,69,83,150,4,141,13,136,18,150,4,100,54,18,133,42,108,132,21,96,51,128,19,6,110,...,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,74,0,0,124,5,0,0,235,235,235,40,195,226,903,2.8
27226239,2017,2,27,2702306,2,2,1,1,49,67,92,26,76,42,94,22,56,58,115,2,108,9,102,15,115,2,80,37,20,94,33,81,98,17,83,30,97,17,5,50,...,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,207,0,0,194,0,0,0,401,401,401,38,363,401,976,3.1
27226468,2017,2,27,2708303,2,2,1,1,58,90,102,49,90,60,105,44,78,68,141,9,133,15,118,31,143,4,98,49,29,122,38,108,126,23,116,35,124,20,5,90,...,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,144,0,0,38,0,0,0,235,235,235,45,190,228,597,4.1
27226808,2017,2,27,2707602,2,2,1,1,32,46,60,18,50,28,48,29,33,43,77,1,69,9,63,15,77,1,55,23,11,67,14,64,64,13,59,19,63,9,6,54,...,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,33,0,0,92,6,0,0,168,168,168,79,89,125,397,3.9


In [ ]:
dados_escola.dtypes[dados_escola.dtypes == np.float64]

Ideb-2017    float64
dtype: object

In [ ]:
dataset['ID_DEPENDENCIA_ADM'].unique()

array([2, 1])

In [ ]:
#dataset.to_csv('C:/Users/maril/Documents/ufs/TCC e Pesquisa Ideb/variaveis_derivadas_escola_estudante_5.0.csv')
dataset.to_csv('/content/drive/MyDrive/TCC/variaveis_derivadas_escola_estudante_5.0.csv')

### Discretização dos rótulos, nota do Ideb

**Discretização:** É o processo de transferência de funções contínuas, modelos, variáveis e equações em contrapartes discretas.

In [ ]:
# São três classes média, acima da média e abaixo da média
def discretizacao(ideb, mean_ideb):
    classe = []
    for nota in ideb['Ideb-2017']:
        if (mean_ideb == nota):
            classe.append('média')
        elif (mean_ideb < nota):
            classe.append('acima da média')
        else:
            classe.append('abaixo da média')
    return classe

In [ ]:
#discretizando as notas do Ideb

Ideb = dataset[['Ideb-2017']]

mean_Ideb = round(Ideb.mean(),1)

classe_ideb = discretizacao(Ideb, mean_Ideb['Ideb-2017'])

Ideb.insert(1, 'classe-Ideb', classe_ideb)

Ideb

,Ideb-2017,classe-Ideb
27000095,3.4,média
27001237,3.9,acima da média
27002020,3.3,abaixo da média
27002063,3.8,acima da média
27002950,3.5,acima da média
...,...,...
27226131,2.8,abaixo da média
27226239,3.1,abaixo da média
27226468,4.1,acima da média
27226808,3.9,acima da média


In [ ]:
#Variavel a ser prevista
y = Ideb['classe-Ideb']

**Seleção de Atributos:**  É um passo da fase de pré-processamento que tem a finalidade de selecionar o melhor subconjunto de atributos que pode substituir o conjunto de dados original. 

https://www.analyticsvidhya.com/blog/2016/12/introduction-to-feature-selection-methods-with-an-example-or-how-to-select-the-right-variables/#:~:text=Embedded%20methods%20combine%20the%20qualities,penalization%20functions%20to%20reduce%20overfitting.

Abordagens:

- Filtro: a relevância dos atributos é avaliada independentemente – não
envolve o uso de um método de AM ;

- Wrapper (Envólucro): Avalia o subconjunto de atributos arti culado ao método de AM que realizará a indução do conceito;

- Embedded (Embutidos): Os métodos incorporados combinam as qualidades dos métodos de filtro e wrapper. Ele é implementado por algoritmos que possuem seus próprios métodos de seleção de recursos integrados.

In [ ]:
#Filter based
# f_classif: ANOVA F-value between label/feature for classification tasks.
#ANOVA stands for Analysis of variance. It is similar to LDA except for the fact that it is operated using one or more categorical 
#independent features and one continuous dependent feature. It provides a statistical test 
#of whether the means of several groups are equal or not.
def selecao_atributos(X, y, quant):
    select = SelectKBest(score_func=f_classif, k=quant)
    #select = SelectKBest(score_func=chi2, k=quant)
    z = select.fit_transform(X, y)
    
    filter = select.get_support(indices=True)
    features = array(X.columns)
    
    return X.iloc[:, filter]

In [ ]:
#Wrapper-based
#from sklearn.cluster import KMeans

def wrapper_based(X, X_transform, y, quant):

  #select_features = RFE(estimator=KMeans(n_clusters=3), n_features_to_select=100)
  select_features = RFE(estimator=SVC(kernel="linear"), n_features_to_select=100)
  '''select_features = RFECV(
      estimator=SVC(kernel="linear"),
      min_features_to_select= quant,
      cv = 5
  )'''
  select_features.fit(X_transform, y)

  filter = select_features.get_support(indices=True)
  return X.iloc[:, filter]

In [ ]:
# Embedded-based

def embedded_based(X, X_transform, y, quant):

  select_features = SelectFromModel(LogisticRegression(), max_features=quant)
  #select_features = SelectFromModel(DecisionTreeClassifier(), max_features=quant)
  #select_features = SelectFromModel(SVC(kernel="rbf"), max_features=quant)
  #select_features = SelectFromModel(RandomForestClassifier(), max_features=quant)
  select_features.fit(X_transform, y)

  filter = select_features.get_support(indices=True)

  return X.iloc[:, filter]

**Métricas de Avaliação**


In [ ]:
def acuracia(resultados, classes):
    #resultados = cross_val_predict(modelo, textos, classes, cv=10)  
    return 'Acurácia do modelo: {} %'.format(round(metrics.accuracy_score(classes, resultados)*100, 2))

**Normalização:** é mudar os valores das colunas numéricas no conjunto de dados para usar uma escala comum, sem distorcer as diferenças nos intervalos de valores nem perder informações.

- **Normalização MinMax:** Para cada recurso, o valor mínimo desse recurso é transformado em 0, o valor máximo é transformado em 1 e todos os outros valores são transformados em um decimal entre 0 e 1.

In [ ]:
def minmax(data):
    scaler = MinMaxScaler()
    data_scaler = scaler.fit_transform(data)
    return data_scaler

## Criação dos modelos

**Serão feitas três análises:** Apenas com as variáveis socioeconômicas, com as variáveis de escolas e professores e depois com o dataset completo

Para cada uma serão criados modelos com todos os atributos e também com seleção de atributos, a fim de verificar o de maior acurácia

In [ ]:
def inicializar_modelos():
    naive = MultinomialNB()
    random = RandomForestClassifier(n_estimators=150)
    decision_tree = DecisionTreeClassifier()
    svm = SVC(kernel="linear")
    r_logistic = LogisticRegression(max_iter=250)
    knn = KNeighborsClassifier(n_neighbors=3)
    return naive, random, decision_tree, svm, r_logistic, knn

#### -Primeira análise: variáveis socioeconômicas derivadas da prova SAEB**

In [ ]:
#Foi retirada colunas do 'ID_PROVA_BRASIL', 'ID_REGIAO', 'ID_UF'
columns = dataset.columns[3:234]

columns

Index(['ID_MUNICIPIO', 'ID_AREA', 'ID_DEPENDENCIA_ADM', 'ID_LOCALIZACAO',
       'IN_SITUACAO_CENSO', 'TX_RESP_Q001_A', 'TX_RESP_Q001_B',
       'TX_RESP_Q020_A', 'TX_RESP_Q020_B', 'TX_RESP_Q021_A',
       ...
       'TX_RESP_Q003_A', 'TX_RESP_Q003_B', 'TX_RESP_Q003_C', 'TX_RESP_Q003_D',
       'TX_RESP_Q003_E', 'TX_RESP_Q003_F', 'TX_RESP_Q003_G', 'TX_RESP_Q003_H',
       'TX_RESP_Q003_I', 'TX_RESP_Q003_J'],
      dtype='object', length=231)

In [ ]:
X =  dataset[columns]

In [ ]:
#Questões de sexo, raça, ano, mes de nascimento, conformidade com censo
question = ['TX_RESP_Q001_A', 'TX_RESP_Q001_B', 'TX_RESP_Q002_A', 'TX_RESP_Q002_B', 'TX_RESP_Q002_C', 'TX_RESP_Q002_D', 
            'TX_RESP_Q002_E', 'TX_RESP_Q002_F', 'TX_RESP_Q003_A', 'TX_RESP_Q003_B', 'TX_RESP_Q003_C', 'TX_RESP_Q003_D', 
            'TX_RESP_Q003_E', 'TX_RESP_Q003_F', 'TX_RESP_Q003_G', 'TX_RESP_Q003_H', 'TX_RESP_Q003_I', 'TX_RESP_Q003_J', 
            'TX_RESP_Q004_A', 'TX_RESP_Q004_B', 'TX_RESP_Q004_C', 'TX_RESP_Q004_D', 
            'TX_RESP_Q004_E', 'TX_RESP_Q004_F', 'TX_RESP_Q004_G', 'TX_RESP_Q004_H', 'IN_SITUACAO_CENSO']

delete_at = ['IN_SITUACAO_CENSO']

X.drop(delete_at, axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
filteredColumns = X.dtypes[X.dtypes == np.float]

listOfColumnNames = list(filteredColumns.index)

X[listOfColumnNames]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.


""
27000095
27001237
27002020
27002063
27002950
...
27226131
27226239
27226468
27226808


**Divisão entre conjunto de treino e teste**

In [ ]:
X_scaler = minmax(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaler, y, random_state=50, test_size=0.2)

**Fase de Treinamento e Teste**

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 58.33 %
Naive Bayes: Acurácia do modelo: 50.0 %
Decision Tree: Acurácia do modelo: 45.83 %
SVM: Acurácia do modelo: 41.67 %
Logistic Regression: Acurácia do modelo: 41.67 %
KNN: Acurácia do modelo: 50.0 %


**-Primeira Análise: Aplicando Seleção de Atributos (Filtragem)**

In [ ]:
X_selection = selecao_atributos(X, y, 100)

X_selection

,ID_DEPENDENCIA_ADM,TX_RESP_Q001_B,TX_RESP_Q020_B,TX_RESP_Q021_B,TX_RESP_Q024_B,TX_RESP_Q025_B,TX_RESP_Q027_A,TX_RESP_Q028_A,TX_RESP_Q028_B,TX_RESP_Q029_A,TX_RESP_Q030_A,TX_RESP_Q030_B,TX_RESP_Q031_A,TX_RESP_Q038_B,TX_RESP_Q043_A,TX_RESP_Q044_A,TX_RESP_Q052_A,TX_RESP_Q018_A,TX_RESP_Q018_B,TX_RESP_Q022_B,TX_RESP_Q026_B,TX_RESP_Q026_C,TX_RESP_Q032_A,TX_RESP_Q033_A,TX_RESP_Q034_A,TX_RESP_Q035_C,TX_RESP_Q040_A,TX_RESP_Q041_B,TX_RESP_Q041_C,TX_RESP_Q042_B,TX_RESP_Q042_C,TX_RESP_Q046_A,TX_RESP_Q047_A,TX_RESP_Q048_B,TX_RESP_Q050_B,TX_RESP_Q054_B,TX_RESP_Q055_B,TX_RESP_Q056_B,TX_RESP_Q057_A,TX_RESP_Q058_B,...,TX_RESP_Q010_C,TX_RESP_Q011_A,TX_RESP_Q011_D,TX_RESP_Q012_A,TX_RESP_Q013_A,TX_RESP_Q013_C,TX_RESP_Q013_D,TX_RESP_Q014_B,TX_RESP_Q015_B,TX_RESP_Q015_C,TX_RESP_Q017_B,TX_RESP_Q017_C,TX_RESP_Q017_E,TX_RESP_Q036_A,TX_RESP_Q036_B,TX_RESP_Q036_E,TX_RESP_Q037_A,TX_RESP_Q037_D,TX_RESP_Q037_E,TX_RESP_Q002_C,TX_RESP_Q002_F,TX_RESP_Q016_E,TX_RESP_Q016_F,TX_RESP_Q019_A,TX_RESP_Q019_B,TX_RESP_Q019_G,TX_RESP_Q023_A,TX_RESP_Q023_B,TX_RESP_Q023_G,TX_RESP_Q004_A,TX_RESP_Q004_B,TX_RESP_Q004_E,TX_RESP_Q004_F,TX_RESP_Q004_G,TX_RESP_Q004_H,TX_RESP_Q003_A,TX_RESP_Q003_B,TX_RESP_Q003_C,TX_RESP_Q003_D,TX_RESP_Q003_F
27000095,2,59,22,30,28,50,92,87,8,82,93,1,73,90,27,80,73,86,9,24,48,9,10,9,36,9,94,49,20,7,4,17,25,7,20,3,29,30,5,48,...,0,78,0,85,80,1,0,84,8,35,7,0,1,32,24,15,16,22,14,74,2,21,33,10,32,16,14,28,26,0,1,11,17,4,5,9,6,6,7,11
27001237,2,108,44,58,68,100,177,167,16,162,179,4,133,139,55,146,102,153,30,55,60,19,25,25,64,5,178,52,9,13,5,25,44,25,17,5,33,66,21,40,...,4,114,0,143,145,6,0,159,8,76,16,2,0,55,40,12,34,38,34,122,6,50,54,19,67,22,32,58,34,0,4,25,15,6,9,13,18,15,14,12
27002020,2,60,21,28,29,47,95,88,9,89,91,5,73,80,36,84,61,79,20,34,26,9,18,16,38,4,87,34,10,6,3,36,31,8,10,3,24,28,16,18,...,1,66,0,72,78,2,1,68,8,38,11,2,0,27,22,16,28,15,21,69,5,22,22,6,35,16,12,29,21,1,2,11,13,5,5,10,5,7,8,6
27002063,2,24,13,20,18,26,34,32,4,33,35,1,26,32,14,28,29,33,2,10,13,9,4,3,12,8,36,15,5,2,1,6,12,3,11,1,8,10,4,21,...,0,24,1,25,30,0,0,34,2,14,0,1,0,9,7,5,8,10,7,25,2,12,9,3,15,6,4,17,9,0,0,7,4,1,3,1,1,4,2,4
27002950,2,72,31,52,46,77,134,123,13,115,133,4,98,122,36,93,83,116,22,55,48,14,15,27,45,9,138,29,14,13,6,26,41,9,21,5,23,38,15,35,...,0,101,0,120,112,1,0,116,13,73,7,1,0,41,30,25,39,26,17,83,2,34,37,14,45,20,23,35,36,0,1,13,11,3,10,7,7,16,11,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27226131,2,86,46,57,57,83,150,141,13,136,150,4,100,133,42,132,96,128,19,38,55,30,15,23,45,12,144,40,14,20,3,26,46,29,12,12,41,47,31,27,...,2,117,0,116,107,2,0,130,8,71,8,3,0,39,43,4,33,32,26,88,4,31,33,23,52,14,32,48,24,0,5,17,10,4,11,15,11,15,14,9
27226239,2,67,26,42,22,58,115,108,9,102,115,2,80,94,33,98,83,97,17,58,44,26,18,16,39,6,112,13,9,7,1,16,28,13,9,8,25,51,18,15,...,2,72,0,99,90,1,1,95,2,63,14,4,1,25,24,9,36,19,28,76,1,28,20,5,35,16,6,29,31,0,1,15,2,1,0,10,6,9,6,11
27226468,2,90,49,60,44,68,141,133,15,118,143,4,98,122,38,126,116,124,20,49,50,13,18,12,54,8,144,49,8,12,2,12,22,8,16,5,22,56,11,36,...,1,113,0,124,106,9,0,119,8,68,8,0,0,50,40,10,38,18,26,84,5,38,26,25,53,21,22,47,37,0,12,19,8,1,3,14,16,12,9,14
27226808,2,46,18,28,29,43,77,69,9,63,77,1,55,67,14,64,59,63,9,19,14,7,11,11,27,10,76,26,11,7,3,19,21,2,27,3,9,26,9,40,...,1,61,0,68,65,1,0,65,0,39,3,0,0,19,29,5,24,15,9,54,0,26,11,11,26,11,11,28,20,2,9,8,0,4,5,8,7,10,3,7


In [ ]:
X_scaler = minmax(X_selection)

X_train, X_test, y_train, y_test = train_test_split(X_scaler, y, random_state=50, test_size=0.2)

**Fase de Treinamento e Teste**

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 45.83 %
Naive Bayes: Acurácia do modelo: 54.17 %
Decision Tree: Acurácia do modelo: 41.67 %
SVM: Acurácia do modelo: 45.83 %
Logistic Regression: Acurácia do modelo: 58.33 %
KNN: Acurácia do modelo: 33.33 %


**-Primeira Análise: Aplicando Seleção de Atributos (Embutida)**

In [ ]:
X_embedded = embedded_based(X, X_scaler, y, 100)

X_embedded

,ID_MUNICIPIO,ID_DEPENDENCIA_ADM,TX_RESP_Q020_B,TX_RESP_Q024_A,TX_RESP_Q024_B,TX_RESP_Q031_B,TX_RESP_Q038_B,TX_RESP_Q043_A,TX_RESP_Q052_B,TX_RESP_Q022_A,TX_RESP_Q032_C,TX_RESP_Q033_B,TX_RESP_Q033_C,TX_RESP_Q035_A,TX_RESP_Q035_C,TX_RESP_Q040_A,TX_RESP_Q040_B,TX_RESP_Q041_A,TX_RESP_Q041_B,TX_RESP_Q042_B,TX_RESP_Q047_A,TX_RESP_Q047_B,TX_RESP_Q048_B,TX_RESP_Q048_C,TX_RESP_Q049_A,TX_RESP_Q049_B,TX_RESP_Q050_C,TX_RESP_Q055_A,TX_RESP_Q055_C,TX_RESP_Q056_C,TX_RESP_Q057_A,TX_RESP_Q058_A,TX_RESP_Q058_B,TX_RESP_Q058_C,TX_RESP_Q039_C,TX_RESP_Q039_D
27000095,2700102,2,22,68,28,22,90,27,20,68,22,43,30,50,9,94,1,25,49,7,25,53,7,1,45,35,1,60,2,2,5,32,48,2,32,2
27001237,2701605,2,44,113,68,49,139,55,75,123,41,75,63,108,5,178,3,118,52,13,44,90,25,12,75,62,14,130,4,6,21,112,40,6,53,1
27002020,2705002,2,21,66,29,24,80,36,24,64,13,48,25,59,4,87,0,48,34,6,31,45,8,3,40,36,2,52,0,1,16,55,18,2,28,3
27002063,2705002,2,13,18,18,9,32,14,5,26,14,8,21,13,8,36,0,15,15,2,12,18,3,3,16,12,1,20,2,2,4,10,21,1,15,2
27002950,2706422,2,31,89,46,38,122,36,52,81,31,48,48,73,9,138,0,93,29,13,41,74,9,2,69,45,4,102,2,2,15,79,35,5,15,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27226131,2700706,2,46,96,57,54,133,42,51,110,46,70,55,71,12,144,0,100,40,20,46,93,29,1,71,66,0,103,1,3,31,115,27,0,24,0
27226239,2702306,2,26,94,22,37,94,33,30,50,32,44,46,70,6,112,1,90,13,7,28,70,13,3,48,49,2,79,3,2,18,94,15,1,24,0
27226468,2708303,2,49,105,44,49,122,38,35,90,29,58,62,86,8,144,2,90,49,12,22,99,8,6,60,60,5,113,2,3,11,95,36,3,55,3
27226808,2707602,2,18,48,29,23,67,14,19,54,22,35,29,35,10,76,1,39,26,7,21,49,2,0,31,39,2,66,0,0,9,31,40,2,17,3


In [ ]:
X_scaler_em = minmax(X_embedded)

X_train, X_test, y_train, y_test = train_test_split(X_scaler_em, y, random_state=50, test_size=0.2)

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 62.5 %
Naive Bayes: Acurácia do modelo: 54.17 %
Decision Tree: Acurácia do modelo: 45.83 %
SVM: Acurácia do modelo: 62.5 %
Logistic Regression: Acurácia do modelo: 62.5 %
KNN: Acurácia do modelo: 54.17 %


**-Primeira Análise: Aplicando Seleção de Atributos (Wrapper)**

In [ ]:
X_wrapper = wrapper_based(X, X_scaler, y, 100)

X_wrapper

,ID_MUNICIPIO,ID_AREA,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,TX_RESP_Q001_A,TX_RESP_Q001_B,TX_RESP_Q020_A,TX_RESP_Q020_B,TX_RESP_Q021_A,TX_RESP_Q021_B,TX_RESP_Q024_A,TX_RESP_Q024_B,TX_RESP_Q025_A,TX_RESP_Q025_B,TX_RESP_Q027_A,TX_RESP_Q027_B,TX_RESP_Q028_A,TX_RESP_Q028_B,TX_RESP_Q029_A,TX_RESP_Q029_B,TX_RESP_Q030_A,TX_RESP_Q030_B,TX_RESP_Q031_A,TX_RESP_Q031_B,TX_RESP_Q038_A,TX_RESP_Q038_B,TX_RESP_Q043_A,TX_RESP_Q043_B,TX_RESP_Q044_A,TX_RESP_Q044_B,TX_RESP_Q052_A,TX_RESP_Q052_B,TX_RESP_Q018_A,TX_RESP_Q018_B,TX_RESP_Q018_C,TX_RESP_Q022_A,TX_RESP_Q022_B,TX_RESP_Q022_C,TX_RESP_Q026_A,TX_RESP_Q026_B,...,TX_RESP_Q042_B,TX_RESP_Q042_C,TX_RESP_Q046_A,TX_RESP_Q046_B,TX_RESP_Q046_C,TX_RESP_Q047_A,TX_RESP_Q047_B,TX_RESP_Q047_C,TX_RESP_Q048_A,TX_RESP_Q048_B,TX_RESP_Q048_C,TX_RESP_Q049_A,TX_RESP_Q049_B,TX_RESP_Q049_C,TX_RESP_Q050_A,TX_RESP_Q050_B,TX_RESP_Q050_C,TX_RESP_Q054_A,TX_RESP_Q054_B,TX_RESP_Q054_C,TX_RESP_Q055_A,TX_RESP_Q055_B,TX_RESP_Q055_C,TX_RESP_Q056_A,TX_RESP_Q056_B,TX_RESP_Q056_C,TX_RESP_Q057_A,TX_RESP_Q057_B,TX_RESP_Q057_C,TX_RESP_Q058_A,TX_RESP_Q058_B,TX_RESP_Q058_C,TX_RESP_Q039_A,TX_RESP_Q039_B,TX_RESP_Q039_C,TX_RESP_Q039_D,TX_RESP_Q045_A,TX_RESP_Q045_B,TX_RESP_Q045_C,TX_RESP_Q045_D
27000095,2700102,2,2,1,38,59,75,22,68,30,68,28,40,50,92,3,87,8,82,13,93,1,73,22,6,90,27,67,80,16,73,20,86,9,2,68,24,4,38,48,...,7,4,17,61,3,25,53,3,84,7,1,45,35,2,63,20,1,78,3,2,60,29,2,48,30,2,5,71,4,32,48,2,15,46,32,2,70,24,1,1
27001237,2701605,2,2,1,76,108,144,44,130,58,113,68,77,100,177,6,167,16,162,21,179,4,133,49,43,139,55,119,146,36,102,75,153,30,4,123,55,5,103,60,...,13,5,25,111,19,44,90,17,121,25,12,75,62,16,133,17,14,154,5,5,130,33,4,80,66,6,21,121,8,112,40,6,40,88,53,1,104,61,7,9
27002020,2705002,2,2,1,41,60,78,21,70,28,66,29,49,47,95,2,88,9,89,8,91,5,73,24,13,80,36,52,84,8,61,24,79,20,1,64,34,1,62,26,...,6,3,36,43,3,31,45,5,73,8,3,40,36,3,72,10,2,77,3,0,52,24,0,48,28,1,16,55,1,55,18,2,28,35,28,3,73,16,3,2
27002063,2705002,2,2,2,12,24,22,13,15,20,18,18,9,26,34,2,32,4,33,3,35,1,26,9,2,32,14,21,28,7,29,5,33,2,0,26,10,0,13,13,...,2,1,6,23,3,12,18,2,29,3,3,16,12,3,21,11,1,32,1,1,20,8,2,19,10,2,4,26,1,10,21,1,0,18,15,2,23,10,2,1
27002950,2706422,2,2,1,70,72,109,31,88,52,89,46,57,77,134,3,123,13,115,22,133,4,98,38,14,122,36,102,93,46,83,52,116,22,3,81,55,3,76,48,...,13,6,26,91,4,41,74,4,122,9,2,69,45,4,97,21,4,119,5,2,102,23,2,77,38,2,15,97,4,79,35,5,21,95,15,7,80,55,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27226131,2700706,2,2,1,64,86,108,46,97,57,96,57,69,83,150,4,141,13,136,18,150,4,100,54,18,133,42,108,132,21,96,51,128,19,6,110,38,6,69,55,...,20,3,26,111,5,46,93,7,116,29,1,71,66,3,133,12,0,138,12,0,103,41,1,92,47,3,31,109,2,115,27,0,60,68,24,0,83,66,3,1
27226239,2702306,2,2,1,49,67,92,26,76,42,94,22,56,58,115,2,108,9,102,15,115,2,80,37,20,94,33,81,98,17,83,30,97,17,5,50,58,9,46,44,...,7,1,16,81,4,28,70,4,91,13,3,48,49,3,101,9,2,100,8,2,79,25,3,48,51,2,18,76,4,94,15,1,49,43,24,0,48,62,2,4
27226468,2708303,2,2,1,58,90,102,49,90,60,105,44,78,68,141,9,133,15,118,31,143,4,98,49,29,122,38,108,126,23,116,35,124,20,5,90,49,12,88,50,...,12,2,12,106,12,22,99,7,129,8,6,60,60,7,116,16,5,132,5,2,113,22,2,73,56,3,11,117,3,95,36,3,20,73,55,3,71,68,5,7
27226808,2707602,2,2,1,32,46,60,18,50,28,48,29,33,43,77,1,69,9,63,15,77,1,55,23,11,67,14,64,64,13,59,19,63,9,6,54,19,5,56,14,...,7,3,19,50,3,21,49,2,71,2,0,31,39,2,46,27,2,72,3,0,66,9,0,46,26,0,9,62,1,31,40,2,10,48,17,3,56,19,3,0


In [ ]:
X_scaler_w = minmax(X_wrapper)

X_train, X_test, y_train, y_test = train_test_split(X_scaler_w, y, random_state=50, test_size=0.2)

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 50.0 %
Naive Bayes: Acurácia do modelo: 62.5 %
Decision Tree: Acurácia do modelo: 41.67 %
SVM: Acurácia do modelo: 58.33 %
Logistic Regression: Acurácia do modelo: 58.33 %
KNN: Acurácia do modelo: 41.67 %


#### -Segunda análise: Análise com o dataset completo

In [ ]:
#Lembrar de tratar datas

'''question = ['TX_RESP_Q001_A', 'TX_RESP_Q001_B', 'TX_RESP_Q002_A', 'TX_RESP_Q002_B', 'TX_RESP_Q002_C', 'TX_RESP_Q002_D', 
            'TX_RESP_Q002_E', 'TX_RESP_Q002_F', 'TX_RESP_Q003_A', 'TX_RESP_Q003_B', 'TX_RESP_Q003_C', 'TX_RESP_Q003_D', 
            'TX_RESP_Q003_E', 'TX_RESP_Q003_F', 'TX_RESP_Q003_G', 'TX_RESP_Q003_H', 'TX_RESP_Q003_I', 'TX_RESP_Q003_J', 
            'TX_RESP_Q003_L', 'TX_RESP_Q004_A', 'TX_RESP_Q004_B', 'TX_RESP_Q004_C', 'TX_RESP_Q004_D', 'CO_REGIAO',
            'TP_SITUACAO_FUNCIONAMENTO', 'TP_REGULAMENTACAO',
            'CO_MESORREGIAO', 'CO_MICRORREGIAO', 'CO_DISTRITO', 'TP_LOCALIZACAO',  'CO_ORGAO_REGIONAL', 'TP_DEPENDENCIA',
            'TX_RESP_Q004_E', 'TX_RESP_Q004_F', 'TX_RESP_Q004_G', 'TX_RESP_Q004_H', 'ID_PROVA_BRASIL', 'ID_REGIAO', 'ID_UF', 'ID_MUNICIPIO', 
            'DT_ANO_LETIVO_INICIO', 'DT_ANO_LETIVO_TERMINO', 'IN_SITUACAO_CENSO', 'NU_ANO_CENSO', 'CO_UF', 'CO_MUNICIPIO', 'Ideb-2017']'''

'''delete = ['CO_MESORREGIAO', 'CO_MICRORREGIAO', 'CO_DISTRITO', 'TP_LOCALIZACAO',  'CO_ORGAO_REGIONAL', 'TP_DEPENDENCIA',
          'ID_PROVA_BRASIL', 'ID_REGIAO', 'ID_UF', 'ID_MUNICIPIO', 'IN_SITUACAO_CENSO', 'Ideb-2017']'''


#X = dataset.drop(delete, axis=1)

'''X = dataset.drop(['Ideb-2017','IN_SITUACAO_CENSO', 'TX_RESP_Q001_A', 'TX_RESP_Q001_B', 'TX_RESP_Q002_A', 'TX_RESP_Q002_B', 'TX_RESP_Q002_C', 'TX_RESP_Q002_D', 
            'TX_RESP_Q002_E', 'TX_RESP_Q002_F', 'TX_RESP_Q003_A', 'TX_RESP_Q003_B', 'TX_RESP_Q003_C', 'TX_RESP_Q003_D', 
            'TX_RESP_Q003_E', 'TX_RESP_Q003_F', 'TX_RESP_Q003_G', 'TX_RESP_Q003_H', 'TX_RESP_Q003_I', 'TX_RESP_Q003_J', 
            'TX_RESP_Q003_L', 'TX_RESP_Q004_A', 'TX_RESP_Q004_B', 'TX_RESP_Q004_C', 'TX_RESP_Q004_D',], axis=1)'''

X = dataset.drop(['Ideb-2017','IN_SITUACAO_CENSO'], axis=1)

In [ ]:
list(X.columns)

['ID_PROVA_BRASIL',
 'ID_REGIAO',
 'ID_UF',
 'ID_MUNICIPIO',
 'ID_AREA',
 'ID_DEPENDENCIA_ADM',
 'ID_LOCALIZACAO',
 'TX_RESP_Q001_A',
 'TX_RESP_Q001_B',
 'TX_RESP_Q020_A',
 'TX_RESP_Q020_B',
 'TX_RESP_Q021_A',
 'TX_RESP_Q021_B',
 'TX_RESP_Q024_A',
 'TX_RESP_Q024_B',
 'TX_RESP_Q025_A',
 'TX_RESP_Q025_B',
 'TX_RESP_Q027_A',
 'TX_RESP_Q027_B',
 'TX_RESP_Q028_A',
 'TX_RESP_Q028_B',
 'TX_RESP_Q029_A',
 'TX_RESP_Q029_B',
 'TX_RESP_Q030_A',
 'TX_RESP_Q030_B',
 'TX_RESP_Q031_A',
 'TX_RESP_Q031_B',
 'TX_RESP_Q038_A',
 'TX_RESP_Q038_B',
 'TX_RESP_Q043_A',
 'TX_RESP_Q043_B',
 'TX_RESP_Q044_A',
 'TX_RESP_Q044_B',
 'TX_RESP_Q052_A',
 'TX_RESP_Q052_B',
 'TX_RESP_Q018_A',
 'TX_RESP_Q018_B',
 'TX_RESP_Q018_C',
 'TX_RESP_Q022_A',
 'TX_RESP_Q022_B',
 'TX_RESP_Q022_C',
 'TX_RESP_Q026_A',
 'TX_RESP_Q026_B',
 'TX_RESP_Q026_C',
 'TX_RESP_Q032_A',
 'TX_RESP_Q032_B',
 'TX_RESP_Q032_C',
 'TX_RESP_Q033_A',
 'TX_RESP_Q033_B',
 'TX_RESP_Q033_C',
 'TX_RESP_Q034_A',
 'TX_RESP_Q034_B',
 'TX_RESP_Q034_C',
 'TX_RESP_Q

In [ ]:
X

,ID_PROVA_BRASIL,ID_REGIAO,ID_UF,ID_MUNICIPIO,ID_AREA,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,TX_RESP_Q001_A,TX_RESP_Q001_B,TX_RESP_Q020_A,TX_RESP_Q020_B,TX_RESP_Q021_A,TX_RESP_Q021_B,TX_RESP_Q024_A,TX_RESP_Q024_B,TX_RESP_Q025_A,TX_RESP_Q025_B,TX_RESP_Q027_A,TX_RESP_Q027_B,TX_RESP_Q028_A,TX_RESP_Q028_B,TX_RESP_Q029_A,TX_RESP_Q029_B,TX_RESP_Q030_A,TX_RESP_Q030_B,TX_RESP_Q031_A,TX_RESP_Q031_B,TX_RESP_Q038_A,TX_RESP_Q038_B,TX_RESP_Q043_A,TX_RESP_Q043_B,TX_RESP_Q044_A,TX_RESP_Q044_B,TX_RESP_Q052_A,TX_RESP_Q052_B,TX_RESP_Q018_A,TX_RESP_Q018_B,TX_RESP_Q018_C,TX_RESP_Q022_A,TX_RESP_Q022_B,...,IN_ESPECIAL_EXCLUSIVA,IN_REGULAR,IN_EJA,IN_PROFISSIONALIZANTE,IN_COMUM_CRECHE,IN_COMUM_PRE,IN_COMUM_FUND_AI,IN_COMUM_FUND_AF,IN_COMUM_MEDIO_MEDIO,IN_COMUM_MEDIO_INTEGRADO,IN_COMUM_MEDIO_NORMAL,IN_ESP_EXCLUSIVA_CRECHE,IN_ESP_EXCLUSIVA_PRE,IN_ESP_EXCLUSIVA_FUND_AI,IN_ESP_EXCLUSIVA_FUND_AF,IN_ESP_EXCLUSIVA_MEDIO_MEDIO,IN_ESP_EXCLUSIVA_MEDIO_INTEGR,IN_ESP_EXCLUSIVA_MEDIO_NORMAL,IN_COMUM_EJA_FUND,IN_COMUM_EJA_MEDIO,IN_COMUM_EJA_PROF,IN_ESP_EXCLUSIVA_EJA_FUND,IN_ESP_EXCLUSIVA_EJA_MEDIO,IN_ESP_EXCLUSIVA_EJA_PROF,IN_COMUM_PROF,IN_ESP_EXCLUSIVA_PROF,QtdeDocentesEsp,QtdeDocentesMest,QtdeDocentesDout,QtdeDocentesSemPos,QtdeDocentesFCEM,QtdeDocentesDivSocCulta,QtdeDocentesAtendEsp,QtdeDocentesTotal,QtdeTurmasProf,QtdeDocentes,asQtdeDocentesEfetivos,QtdeDocentesTempo,QtdeDocentesComLicenciatura,QtdeAlunosTotal
27000095,2017,2,27,2700102,2,2,1,38,59,75,22,68,30,68,28,40,50,92,3,87,8,82,13,93,1,73,22,6,90,27,67,80,16,73,20,86,9,2,68,24,...,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,62,0,0,103,0,0,0,172,172,172,24,148,172,508
27001237,2017,2,27,2701605,2,2,1,76,108,144,44,130,58,113,68,77,100,177,6,167,16,162,21,179,4,133,49,43,139,55,119,146,36,102,75,153,30,4,123,55,...,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,111,0,0,89,31,0,0,250,250,250,11,225,250,792
27002020,2017,2,27,2705002,2,2,1,41,60,78,21,70,28,66,29,49,47,95,2,88,9,89,8,91,5,73,24,13,80,36,52,84,8,61,24,79,20,1,64,34,...,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,100,0,0,86,0,0,0,222,222,222,5,217,222,652
27002063,2017,2,27,2705002,2,2,2,12,24,22,13,15,20,18,18,9,26,34,2,32,4,33,3,35,1,26,9,2,32,14,21,28,7,29,5,33,2,0,26,10,...,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,20,0,0,43,0,0,0,73,73,73,13,40,63,309
27002950,2017,2,27,2706422,2,2,1,70,72,109,31,88,52,89,46,57,77,134,3,123,13,115,22,133,4,98,38,14,122,36,102,93,46,83,52,116,22,3,81,55,...,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,72,7,0,36,0,0,0,121,121,121,37,84,117,442
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27226131,2017,2,27,2700706,2,2,1,64,86,108,46,97,57,96,57,69,83,150,4,141,13,136,18,150,4,100,54,18,133,42,108,132,21,96,51,128,19,6,110,38,...,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,74,0,0,124,5,0,0,235,235,235,40,195,226,903
27226239,2017,2,27,2702306,2,2,1,49,67,92,26,76,42,94,22,56,58,115,2,108,9,102,15,115,2,80,37,20,94,33,81,98,17,83,30,97,17,5,50,58,...,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,207,0,0,194,0,0,0,401,401,401,38,363,401,976
27226468,2017,2,27,2708303,2,2,1,58,90,102,49,90,60,105,44,78,68,141,9,133,15,118,31,143,4,98,49,29,122,38,108,126,23,116,35,124,20,5,90,49,...,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,144,0,0,38,0,0,0,235,235,235,45,190,228,597
27226808,2017,2,27,2707602,2,2,1,32,46,60,18,50,28,48,29,33,43,77,1,69,9,63,15,77,1,55,23,11,67,14,64,64,13,59,19,63,9,6,54,19,...,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,33,0,0,92,6,0,0,168,168,168,79,89,125,397


In [ ]:
filteredColumns = X.dtypes[X.dtypes == np.float64]

listOfColumnNames = list(filteredColumns.index)

#X[listOfColumnNames].fillna(-1, inplace=True)

#X[listOfColumnNames] = X[listOfColumnNames].astype(int)

X[listOfColumnNames].isnull().sum()

Series([], dtype: float64)

In [ ]:
X_transform = minmax(X)

X_train, X_test, y_train, y_test = train_test_split(X_transform, y, random_state=50, test_size=0.2)

**Fase de Treinamento e Teste**

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 50.0 %
Naive Bayes: Acurácia do modelo: 62.5 %
Decision Tree: Acurácia do modelo: 54.17 %
SVM: Acurácia do modelo: 41.67 %
Logistic Regression: Acurácia do modelo: 54.17 %
KNN: Acurácia do modelo: 50.0 %


**-Segunda Análise: Aplicando Seleção de Atributos (Embutida)**

In [ ]:
X_embedded = embedded_based(X, X_transform, y, 100)

X_embedded

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


,TX_RESP_Q030_B,TX_RESP_Q026_C,TX_RESP_Q035_C,TX_RESP_Q040_B,TX_RESP_Q040_C,TX_RESP_Q041_C,TX_RESP_Q042_B,TX_RESP_Q042_C,TX_RESP_Q046_A,TX_RESP_Q046_C,TX_RESP_Q054_B,TX_RESP_Q057_C,TX_RESP_Q051_C,TX_RESP_Q053_C,TX_RESP_Q059_C,TX_RESP_Q059_D,TX_RESP_Q006_C,TX_RESP_Q006_D,TX_RESP_Q006_E,TX_RESP_Q007_A,TX_RESP_Q007_C,TX_RESP_Q007_D,TX_RESP_Q008_A,TX_RESP_Q008_C,TX_RESP_Q008_D,TX_RESP_Q008_E,TX_RESP_Q009_C,TX_RESP_Q010_C,TX_RESP_Q011_E,TX_RESP_Q012_D,TX_RESP_Q013_C,TX_RESP_Q014_A,TX_RESP_Q017_B,TX_RESP_Q017_C,TX_RESP_Q017_D,TX_RESP_Q017_E,TX_RESP_Q036_C,TX_RESP_Q036_E,TX_RESP_Q037_E,TX_RESP_Q002_F,...,IN_LABORATORIO_CIENCIAS,IN_SALA_ATENDIMENTO_ESPECIAL,IN_QUADRA_ESPORTES_DESCOBERTA,IN_QUADRA_ESPORTES,IN_BIBLIOTECA,IN_BIBLIOTECA_SALA_LEITURA,IN_BANHEIRO_FORA_PREDIO,IN_BANHEIRO_DENTRO_PREDIO,IN_BANHEIRO_PNE,IN_REFEITORIO,IN_ALMOXARIFADO,IN_AUDITORIO,IN_PATIO_DESCOBERTO,IN_AREA_VERDE,IN_LAVANDERIA,NU_SALAS_EXISTENTES,IN_EQUIP_VIDEOCASSETE,IN_EQUIP_DVD,IN_EQUIP_PARABOLICA,IN_EQUIP_COPIADORA,IN_EQUIP_IMPRESSORA_MULT,IN_EQUIP_SOM,IN_EQUIP_MULTIMIDIA,IN_EQUIP_FAX,IN_EQUIP_FOTO,NU_EQUIP_IMPRESSORA,NU_EQUIP_FAX,NU_EQUIP_FOTO,IN_INTERNET,IN_BANDA_LARGA,TP_AEE,TP_ATIVIDADE_COMPLEMENTAR,IN_FINAL_SEMANA,IN_PROFISSIONALIZANTE,IN_COMUM_FUND_AI,IN_COMUM_FUND_AF,IN_COMUM_MEDIO_NORMAL,IN_COMUM_EJA_FUND,IN_COMUM_EJA_MEDIO,QtdeDocentesFCEM
27000095,1,9,9,1,1,20,7,4,17,3,3,4,0,1,0,2,3,0,1,27,3,0,9,1,0,1,2,0,0,1,1,4,7,0,1,1,12,15,14,2,...,1,0,0,0,0,1,0,1,1,0,1,1,0,0,0,10,0,1,0,1,0,1,1,0,0,2,0,0,1,1,0,0,0,0,0,0,0,0,1,0
27001237,4,19,5,3,2,9,13,5,25,19,5,8,6,7,3,6,15,0,0,60,5,2,4,13,0,0,1,4,1,2,6,4,16,2,1,0,27,12,34,6,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,6,0,1,1,1,1,0,1,0,1,2,0,1,0,0,0,0,1,0,0,0,0,0,1,31
27002020,5,9,4,0,7,10,6,3,36,3,3,1,1,3,0,1,6,1,0,46,11,0,5,9,0,0,3,1,0,0,2,8,11,2,0,0,9,16,21,5,...,0,0,0,0,1,1,0,1,1,0,1,0,0,0,0,7,0,1,0,1,0,1,1,0,0,3,0,0,1,1,0,0,0,0,0,0,0,0,1,0
27002063,1,9,8,0,0,5,2,1,6,3,1,1,1,2,0,2,2,0,0,15,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,5,5,7,2,...,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,7,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
27002950,4,14,9,0,2,14,13,6,26,4,5,4,0,3,8,3,4,0,0,55,4,1,8,4,0,0,1,0,0,1,1,7,7,1,0,0,23,25,17,2,...,0,0,0,0,1,1,0,1,1,0,1,0,1,0,0,6,0,1,0,0,1,1,1,0,1,1,0,1,1,1,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27226131,4,30,12,0,10,14,20,3,26,5,12,2,0,10,2,1,3,0,0,69,1,0,3,4,0,0,1,2,0,0,2,1,8,3,0,0,22,4,26,4,...,1,0,0,0,1,1,0,1,1,1,1,1,1,0,0,10,0,1,0,1,0,1,1,1,1,2,1,2,1,1,0,0,0,0,0,0,0,0,1,5
27226239,2,26,6,1,4,9,7,1,16,4,8,4,2,5,2,1,6,2,0,44,6,1,1,6,1,0,2,2,0,0,1,0,14,4,0,1,22,9,28,1,...,1,1,0,0,1,1,0,1,1,1,1,1,1,1,0,15,0,1,0,1,0,1,1,0,1,4,0,3,1,1,0,0,0,0,0,0,0,0,1,0
27226468,4,13,8,2,4,8,12,2,12,12,5,3,3,6,3,3,10,0,0,45,4,0,1,12,0,0,2,1,0,1,9,4,8,0,1,0,18,10,26,5,...,1,1,0,1,1,1,0,1,1,1,1,1,0,0,0,10,0,0,0,1,1,1,1,0,1,2,0,1,1,1,1,0,0,0,0,0,0,0,1,0
27226808,1,7,10,1,1,11,7,3,19,3,3,1,1,3,0,0,4,0,0,22,1,0,0,5,0,0,1,1,0,0,1,1,3,0,1,0,9,5,9,0,...,1,0,0,1,1,1,0,1,1,1,1,1,0,0,0,10,0,1,1,0,0,1,1,1,1,3,1,1,1,0,0,0,0,0,0,0,0,0,1,6


In [ ]:
X_transform_em = minmax(X_embedded)

X_train, X_test, y_train, y_test = train_test_split(X_transform_em, y, random_state=50, test_size=0.2)

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 66.67 %
Naive Bayes: Acurácia do modelo: 70.83 %
Decision Tree: Acurácia do modelo: 58.33 %
SVM: Acurácia do modelo: 45.83 %
Logistic Regression: Acurácia do modelo: 54.17 %
KNN: Acurácia do modelo: 62.5 %


**-Segunda Análise: Aplicando Seleção de Atributos (Wrapper)**

In [ ]:
X_wrapper = wrapper_based(X, X_transform, y, 100)

X_wrapper

,TX_RESP_Q030_B,TX_RESP_Q026_C,TX_RESP_Q034_C,TX_RESP_Q035_C,TX_RESP_Q040_B,TX_RESP_Q040_C,TX_RESP_Q041_C,TX_RESP_Q042_C,TX_RESP_Q046_A,TX_RESP_Q046_C,TX_RESP_Q054_B,TX_RESP_Q055_A,TX_RESP_Q055_C,TX_RESP_Q057_C,TX_RESP_Q058_C,TX_RESP_Q051_C,TX_RESP_Q053_C,TX_RESP_Q059_C,TX_RESP_Q059_D,TX_RESP_Q005_E,TX_RESP_Q006_C,TX_RESP_Q006_D,TX_RESP_Q006_E,TX_RESP_Q007_A,TX_RESP_Q007_C,TX_RESP_Q007_D,TX_RESP_Q008_A,TX_RESP_Q008_C,TX_RESP_Q008_D,TX_RESP_Q008_E,TX_RESP_Q010_C,TX_RESP_Q011_C,TX_RESP_Q011_E,TX_RESP_Q012_D,TX_RESP_Q013_C,TX_RESP_Q014_A,TX_RESP_Q014_C,TX_RESP_Q017_B,TX_RESP_Q017_C,TX_RESP_Q017_D,...,CO_MUNICIPIO,TP_REGULAMENTACAO,IN_LOCAL_FUNC_SALAS_OUTRA_ESC,IN_PREDIO_COMPARTILHADO,IN_AGUA_FILTRADA,IN_AGUA_CACIMBA,IN_ESGOTO_REDE_PUBLICA,IN_LIXO_QUEIMA,IN_SALA_ATENDIMENTO_ESPECIAL,IN_QUADRA_ESPORTES_DESCOBERTA,IN_BIBLIOTECA_SALA_LEITURA,IN_BANHEIRO_FORA_PREDIO,IN_BANHEIRO_DENTRO_PREDIO,IN_BANHEIRO_PNE,IN_AUDITORIO,IN_PATIO_DESCOBERTO,NU_SALAS_EXISTENTES,IN_EQUIP_VIDEOCASSETE,IN_EQUIP_PARABOLICA,IN_EQUIP_COPIADORA,IN_EQUIP_RETROPROJETOR,IN_EQUIP_IMPRESSORA_MULT,IN_EQUIP_SOM,IN_EQUIP_MULTIMIDIA,IN_EQUIP_FAX,IN_EQUIP_FOTO,NU_EQUIP_RETROPROJETOR,NU_EQUIP_IMPRESSORA,NU_EQUIP_FOTO,TP_AEE,TP_ATIVIDADE_COMPLEMENTAR,IN_FINAL_SEMANA,IN_FORMACAO_ALTERNANCIA,IN_EJA,IN_COMUM_FUND_AI,IN_COMUM_FUND_AF,IN_COMUM_MEDIO_NORMAL,IN_COMUM_EJA_FUND,IN_COMUM_EJA_MEDIO,QtdeDocentesFCEM
27000095,1,9,6,9,1,1,20,4,17,3,3,60,2,4,2,0,1,0,2,0,3,0,1,27,3,0,9,1,0,1,0,2,0,1,1,4,9,7,0,1,...,2700102,1,0,0,1,0,1,0,0,0,1,0,1,1,1,0,10,0,0,1,0,0,1,1,0,0,0,2,0,0,0,0,0,1,0,0,0,0,1,0
27001237,4,19,18,5,3,2,9,5,25,19,5,130,4,8,6,6,7,3,6,0,15,0,0,60,5,2,4,13,0,0,4,4,1,2,6,4,22,16,2,1,...,2701605,1,1,0,1,1,0,0,0,0,0,0,1,0,0,0,6,0,1,1,0,1,0,1,0,1,0,2,1,0,0,1,0,1,0,0,0,0,1,31
27002020,5,9,4,4,0,7,10,3,36,3,3,52,0,1,2,1,3,0,1,3,6,1,0,46,11,0,5,9,0,0,1,1,0,0,2,8,17,11,2,0,...,2705002,2,0,0,1,0,0,0,0,0,1,0,1,1,0,0,7,0,0,1,1,0,1,1,0,0,1,3,0,0,0,0,0,1,0,0,0,0,1,0
27002063,1,9,3,8,0,0,5,1,6,3,1,20,2,1,1,1,2,0,2,0,2,0,0,15,1,0,0,0,0,1,0,0,0,0,0,0,2,0,1,0,...,2705002,1,0,0,1,0,1,0,0,0,0,0,1,1,0,1,7,0,1,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0
27002950,4,14,14,9,0,2,14,6,26,4,5,102,2,4,5,0,3,8,3,1,4,0,0,55,4,1,8,4,0,0,0,1,0,1,1,7,15,7,1,0,...,2706422,2,0,1,1,0,0,0,0,0,1,0,1,1,0,1,6,0,0,0,1,1,1,1,0,1,1,1,1,0,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27226131,4,30,18,12,0,10,14,3,26,5,12,103,1,2,0,0,10,2,1,1,3,0,0,69,1,0,3,4,0,0,2,1,0,0,2,1,20,8,3,0,...,2700706,1,0,0,1,1,0,0,0,0,1,0,1,1,1,1,10,0,0,1,1,0,1,1,1,1,1,2,2,0,0,0,0,1,0,0,0,0,1,5
27226239,2,26,20,6,1,4,9,1,16,4,8,79,3,4,1,2,5,2,1,2,6,2,0,44,6,1,1,6,1,0,2,3,0,0,1,0,20,14,4,0,...,2702306,2,0,0,1,0,0,0,1,0,1,0,1,1,1,1,15,0,0,1,0,0,1,1,0,1,0,4,3,0,0,0,0,1,0,0,0,0,1,0
27226468,4,13,11,8,2,4,8,2,12,12,5,113,2,3,3,3,6,3,3,0,10,0,0,45,4,0,1,12,0,0,1,0,0,1,9,4,24,8,0,1,...,2708303,2,0,0,0,0,0,0,1,0,1,0,1,1,1,0,10,0,0,1,0,1,1,1,0,1,0,2,1,1,0,0,0,1,0,0,0,0,1,0
27226808,1,7,10,10,1,1,11,3,19,3,3,66,0,1,2,1,3,0,0,0,4,0,0,22,1,0,0,5,0,0,1,0,0,0,1,1,10,3,0,1,...,2707602,1,0,0,1,0,0,0,0,0,1,0,1,1,1,0,10,0,1,0,1,0,1,1,1,1,1,3,1,0,0,0,0,1,0,0,0,0,1,6


In [ ]:
X_transform_w = minmax(X_wrapper)

X_train, X_test, y_train, y_test = train_test_split(X_transform_w, y, random_state=50, test_size=0.2)

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 62.5 %
Naive Bayes: Acurácia do modelo: 66.67 %
Decision Tree: Acurácia do modelo: 50.0 %
SVM: Acurácia do modelo: 75.0 %
Logistic Regression: Acurácia do modelo: 62.5 %
KNN: Acurácia do modelo: 66.67 %


**-Segunda Análise: Aplicando Seleção de Atributos (Filtragem)**

In [ ]:
X_selection = selecao_atributos(X, y, 100)

X_selection

/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [  0   1   2 234 236 239 242 248 250 251 252 253 254 255 256 266 267 268
 269 272 275 276 277 278 291 292 306 310 325 349 356 357 359 360 363 370
 371 372 373 374 375 376 379 380 381 382 384 390 391] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


,ID_DEPENDENCIA_ADM,TX_RESP_Q001_B,TX_RESP_Q020_B,TX_RESP_Q021_B,TX_RESP_Q024_B,TX_RESP_Q025_B,TX_RESP_Q030_B,TX_RESP_Q043_A,TX_RESP_Q018_B,TX_RESP_Q026_B,TX_RESP_Q026_C,TX_RESP_Q032_A,TX_RESP_Q033_A,TX_RESP_Q034_A,TX_RESP_Q035_C,TX_RESP_Q041_B,TX_RESP_Q041_C,TX_RESP_Q042_B,TX_RESP_Q042_C,TX_RESP_Q046_A,TX_RESP_Q047_A,TX_RESP_Q048_B,TX_RESP_Q054_B,TX_RESP_Q055_B,TX_RESP_Q056_B,TX_RESP_Q057_A,TX_RESP_Q058_B,TX_RESP_Q039_B,TX_RESP_Q051_B,TX_RESP_Q051_C,TX_RESP_Q053_C,TX_RESP_Q059_B,TX_RESP_Q059_C,TX_RESP_Q060_C,TX_RESP_Q005_B,TX_RESP_Q005_E,TX_RESP_Q006_D,TX_RESP_Q006_E,TX_RESP_Q007_A,TX_RESP_Q008_A,...,TX_RESP_Q016_F,TX_RESP_Q019_A,TX_RESP_Q019_B,TX_RESP_Q019_G,TX_RESP_Q023_A,TX_RESP_Q023_B,TX_RESP_Q023_G,TX_RESP_Q004_A,TX_RESP_Q004_B,TX_RESP_Q004_E,TX_RESP_Q004_F,TX_RESP_Q004_G,TX_RESP_Q004_H,TX_RESP_Q003_A,TX_RESP_Q003_B,TX_RESP_Q003_C,TX_RESP_Q003_D,TX_RESP_Q003_F,TX_RESP_Q003_L,CO_ORGAO_REGIONAL,DT_ANO_LETIVO_TERMINO,TP_DEPENDENCIA,TP_REGULAMENTACAO,IN_LOCAL_FUNC_OUTROS,IN_ESGOTO_REDE_PUBLICA,IN_ESGOTO_FOSSA,IN_COZINHA,IN_BANHEIRO_FORA_PREDIO,IN_BANHEIRO_DENTRO_PREDIO,IN_EQUIP_VIDEOCASSETE,IN_EQUIP_PARABOLICA,IN_EQUIP_IMPRESSORA_MULT,NU_EQUIP_VIDEOCASSETE,NU_EQUIP_PARABOLICA,NU_EQUIP_IMPRESSORA,NU_EQUIP_IMPRESSORA_MULT,NU_EQUIP_FOTO,IN_COMUM_MEDIO_MEDIO,IN_COMUM_MEDIO_INTEGRADO,QtdeDocentesTempo
27000095,2,59,22,30,28,50,1,27,9,48,9,10,9,36,9,49,20,7,4,17,25,7,3,29,30,5,48,46,6,0,1,3,0,39,82,0,0,1,27,9,...,33,10,32,16,14,28,26,0,1,11,17,4,5,9,6,6,7,11,7,11,1513296000000000000,2,1,0,1,0,1,0,1,0,0,0,0,0,2,0,0,1,0,148
27001237,2,108,44,58,68,100,4,55,30,60,19,25,25,64,5,52,9,13,5,25,44,25,5,33,66,21,40,88,28,6,7,9,3,76,139,0,0,0,60,4,...,54,19,67,22,32,58,34,0,4,25,15,6,9,13,18,15,14,12,18,11,1513296000000000000,2,1,0,0,1,1,0,1,0,1,1,0,1,2,4,1,1,0,225
27002020,2,60,21,28,29,47,5,36,20,26,9,18,16,38,4,34,10,6,3,36,31,8,3,24,28,16,18,35,2,1,3,10,0,26,72,3,1,0,46,5,...,22,6,35,16,12,29,21,1,2,11,13,5,5,10,5,7,8,6,5,11,1513296000000000000,2,2,0,0,1,1,0,1,0,0,0,0,0,3,0,0,1,0,217
27002063,2,24,13,20,18,26,1,14,2,13,9,4,3,12,8,15,5,2,1,6,12,3,1,8,10,4,21,18,3,1,2,1,0,7,31,0,0,0,15,0,...,9,3,15,6,4,17,9,0,0,7,4,1,3,1,1,4,2,4,1,11,1513296000000000000,2,1,0,1,0,1,0,1,0,1,0,0,1,1,0,0,1,0,40
27002950,2,72,31,52,46,77,4,36,22,48,14,15,27,45,9,29,14,13,6,26,41,9,5,23,38,15,35,95,29,0,3,42,8,65,111,1,0,0,55,8,...,37,14,45,20,23,35,36,0,1,13,11,3,10,7,7,16,11,16,7,11,1513296000000000000,2,2,0,0,1,1,0,1,0,0,1,0,0,1,1,1,1,0,84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27226131,2,86,46,57,57,83,4,42,19,55,30,15,23,45,12,40,14,20,3,26,46,29,12,41,47,31,27,68,17,0,10,12,2,42,130,1,0,0,69,3,...,33,23,52,14,32,48,24,0,5,17,10,4,11,15,11,15,14,9,15,8,1513296000000000000,2,1,0,0,1,1,0,1,0,0,0,0,0,2,0,2,1,0,195
27226239,2,67,26,42,22,58,2,33,17,44,26,18,16,39,6,13,9,7,1,16,28,13,8,25,51,18,15,43,15,2,5,16,2,27,92,2,2,0,44,1,...,20,5,35,16,6,29,31,0,1,15,2,1,0,10,6,9,6,11,9,2,1514505600000000000,2,2,0,0,1,1,0,1,0,0,0,0,0,4,0,3,1,0,363
27226468,2,90,49,60,44,68,4,38,20,50,13,18,12,54,8,49,8,12,2,12,22,8,5,22,56,11,36,73,23,3,6,26,3,35,108,0,0,0,45,1,...,26,25,53,21,22,47,37,0,12,19,8,1,3,14,16,12,9,14,14,7,1514505600000000000,2,2,0,0,1,1,0,1,0,0,1,0,0,2,1,1,1,0,190
27226808,2,46,18,28,29,43,1,14,9,14,7,11,11,27,10,26,11,7,3,19,21,2,3,9,26,9,40,48,11,1,3,9,0,13,64,0,0,0,22,0,...,11,11,26,11,11,28,20,2,9,8,0,4,5,8,7,10,3,7,10,3,1513209600000000000,2,1,0,0,1,1,0,1,0,1,0,0,1,3,0,1,1,0,89


In [ ]:
X_transform = minmax(X_selection)

X_train, X_test, y_train, y_test = train_test_split(X_transform, y, random_state=50, test_size=0.2)

**Fase de Treinamento e Teste**

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_transform)
naive_result = naive.predict(X_transform)
decision_result = decision_tree.predict(X_transform)
svm_result = svm.predict(X_transform)
logistic_result = r_logistic.predict(X_transform)
knn_result = knn.predict(X_transform)

print("Random forest: " + acuracia(random_result, y))
print("Naive Bayes: " + acuracia(naive_result, y))
print("Decision Tree: " + acuracia(decision_result, y))
print("SVM: " + acuracia(svm_result, y))
print("Logistic Regression: "+acuracia(logistic_result, y))
print("KNN: "+acuracia(knn_result, y))

Random forest: Acurácia do modelo: 90.68 %
Naive Bayes: Acurácia do modelo: 61.02 %
Decision Tree: Acurácia do modelo: 88.14 %
SVM: Acurácia do modelo: 82.2 %
Logistic Regression: Acurácia do modelo: 80.51 %
KNN: Acurácia do modelo: 69.49 %


**Segunda Análise: Unindo subconjunto de melhores atributos**

In [ ]:
X_all = pd.concat([X_selection, X_wrapper], axis=1)

X_all = pd.concat([X_all, X_embedded], axis=1)

In [ ]:
#Removendo colunas duplicadas
X_new_all = X_all.T.drop_duplicates().T

In [ ]:
X_transform = minmax(X_new_all)
#X_transform = minmax(X_all)

X_train, X_test, y_train, y_test = train_test_split(X_transform, y, random_state=50, test_size=0.2)

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_transform)
naive_result = naive.predict(X_transform)
decision_result = decision_tree.predict(X_transform)
svm_result = svm.predict(X_transform)
logistic_result = r_logistic.predict(X_transform)
knn_result = knn.predict(X_transform)

print("Random forest: " + acuracia(random_result, y))
print("Naive Bayes: " + acuracia(naive_result, y))
print("Decision Tree: " + acuracia(decision_result, y))
print("SVM: " + acuracia(svm_result, y))
print("Logistic Regression: "+acuracia(logistic_result, y))
print("KNN: "+acuracia(knn_result, y))

Random forest: Acurácia do modelo: 91.53 %
Naive Bayes: Acurácia do modelo: 64.41 %
Decision Tree: Acurácia do modelo: 90.68 %
SVM: Acurácia do modelo: 89.83 %
Logistic Regression: Acurácia do modelo: 90.68 %
KNN: Acurácia do modelo: 66.1 %


Resultado com Todos atributos

- Random forest: Acurácia do modelo: 92.37 %
- Naive Bayes: Acurácia do modelo: 66.1 %
- Decision Tree: Acurácia do modelo: 92.37 %
- SVM: Acurácia do modelo: 89.83 %
- Logistic Regression: Acurácia do modelo: 91.53 %
- KNN: Acurácia do modelo: 66.95 %

**Segunda Análise: Aplicando a técnica merge**

Link da Dissertação: https://www.dcc.ufmg.br/pos/cursos/defesas/1930M.PDF


In [ ]:
#Metodo Merge combina os atributos mais frequentes nos melhores conjuntos de seleção 
#Cada atributo irá apresentar um score de acordo com sua presença em um subconjunto de dados
def metodo_merge(selection, embedded, wrapper, ranking):
  for atributo in selection:
    score = ranking[atributo]
    if (atributo in embedded):
      score += 1
      embedded.remove(atributo)
    if (atributo in wrapper):
      score +=1
      wrapper.remove(atributo)
    ranking[atributo] = score

  for atributo in embedded:
    score = ranking[atributo]
    if (atributo in selection):
      score += 1
      embedded.remove(atributo)
    if (atributo in wrapper):
      score +=1
      wrapper.remove(atributo)
    ranking[atributo] = score

  for atributo in wrapper:
    score = ranking[atributo]
    if (atributo in selection):
      score += 1
      embedded.remove(atributo)
    if (atributo in embedded):
      score +=1
      wrapper.remove(atributo)
    ranking[atributo] = score
 
  return ranking      

In [ ]:
#Gerando Listas com os melhores atributos de cada abordagem
selection = list(X_selection.columns)
embedded = list(X_embedded.columns)
wrapper = list(X_wrapper.columns)

#Criando dicionário
values = [1]*300

all = selection + embedded + wrapper

ranking = dict(zip(all, values))

ranking = metodo_merge(selection, embedded, wrapper, ranking)

#Filtrando os atributos que estão em mais de um subconjunto

dic_aux = ranking.items()

filtered = {key:value for key, value in dic_aux if value > 1}

#Gerando X após o merge

X_merge = dataset[list(filtered.keys())]

In [ ]:
X_transform = minmax(X_merge)

X_train, X_test, y_train, y_test = train_test_split(X_transform, y, random_state=50, test_size=0.2)

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_transform)
naive_result = naive.predict(X_transform)
decision_result = decision_tree.predict(X_transform)
svm_result = svm.predict(X_transform)
logistic_result = r_logistic.predict(X_transform)
knn_result = knn.predict(X_transform)

print("Random forest: " + acuracia(random_result, y))
print("Naive Bayes: " + acuracia(naive_result, y))
print("Decision Tree: " + acuracia(decision_result, y))
print("SVM: " + acuracia(svm_result, y))
print("Logistic Regression: "+acuracia(logistic_result, y))
print("KNN: "+acuracia(knn_result, y))

Random forest: Acurácia do modelo: 92.37 %
Naive Bayes: Acurácia do modelo: 68.64 %
Decision Tree: Acurácia do modelo: 90.68 %
SVM: Acurácia do modelo: 91.53 %
Logistic Regression: Acurácia do modelo: 91.53 %
KNN: Acurácia do modelo: 70.34 %


In [ ]:
dados_para_exportar = pd.DataFrame(columns=X_merge.columns, data=X_merge.values, index=X_merge.index)
dados_para_exportar.insert(len(dados_para_exportar.columns), "Valor Real Ideb", y.to_numpy())
dados_para_exportar.insert(len(dados_para_exportar.columns), "Previsão Ideb", random_result)


dados_para_exportar

,TX_RESP_Q030_B,TX_RESP_Q026_C,TX_RESP_Q035_C,TX_RESP_Q041_C,TX_RESP_Q042_B,TX_RESP_Q042_C,TX_RESP_Q046_A,TX_RESP_Q054_B,TX_RESP_Q051_C,TX_RESP_Q053_C,TX_RESP_Q059_C,TX_RESP_Q005_E,TX_RESP_Q006_D,TX_RESP_Q006_E,TX_RESP_Q007_A,TX_RESP_Q008_A,TX_RESP_Q008_C,TX_RESP_Q010_C,TX_RESP_Q013_C,TX_RESP_Q017_B,TX_RESP_Q017_C,TX_RESP_Q036_E,TX_RESP_Q037_E,TX_RESP_Q002_F,TX_RESP_Q016_F,TX_RESP_Q019_G,TX_RESP_Q004_B,TX_RESP_Q004_H,TX_RESP_Q003_A,TX_RESP_Q003_C,CO_ORGAO_REGIONAL,DT_ANO_LETIVO_TERMINO,TP_REGULAMENTACAO,IN_ESGOTO_REDE_PUBLICA,IN_BANHEIRO_FORA_PREDIO,IN_BANHEIRO_DENTRO_PREDIO,IN_EQUIP_VIDEOCASSETE,IN_EQUIP_PARABOLICA,IN_EQUIP_IMPRESSORA_MULT,NU_EQUIP_IMPRESSORA,...,TX_RESP_Q007_D,TX_RESP_Q008_D,TX_RESP_Q008_E,TX_RESP_Q011_E,TX_RESP_Q012_D,TX_RESP_Q014_A,TX_RESP_Q017_D,TX_RESP_Q017_E,TX_RESP_Q036_C,TX_RESP_Q003_E,TX_RESP_Q003_G,TX_RESP_Q003_H,CO_MICRORREGIAO,IN_PREDIO_COMPARTILHADO,IN_AGUA_FILTRADA,IN_AGUA_CACIMBA,IN_LIXO_QUEIMA,IN_SALA_ATENDIMENTO_ESPECIAL,IN_QUADRA_ESPORTES_DESCOBERTA,IN_BIBLIOTECA_SALA_LEITURA,IN_BANHEIRO_PNE,IN_AUDITORIO,IN_PATIO_DESCOBERTO,NU_SALAS_EXISTENTES,IN_EQUIP_COPIADORA,IN_EQUIP_SOM,IN_EQUIP_MULTIMIDIA,IN_EQUIP_FAX,IN_EQUIP_FOTO,TP_AEE,TP_ATIVIDADE_COMPLEMENTAR,IN_FINAL_SEMANA,IN_COMUM_FUND_AI,IN_COMUM_FUND_AF,IN_COMUM_MEDIO_NORMAL,IN_COMUM_EJA_FUND,IN_COMUM_EJA_MEDIO,QtdeDocentesFCEM,Valor Real Ideb,Previsão Ideb
27000095,1,9,9,20,7,4,17,3,0,1,0,0,0,1,27,9,1,0,1,7,0,15,14,2,33,16,1,5,9,6,11,1513296000000000000,1,1,0,1,0,0,0,2,...,0,0,1,0,1,4,1,1,12,9,9,11,27001,0,1,0,0,0,0,1,1,1,0,10,1,1,1,0,0,0,0,0,0,0,0,0,1,0,média,média
27001237,4,19,5,9,13,5,25,5,6,7,3,0,0,0,60,4,13,4,6,16,2,12,34,6,54,22,4,9,13,15,11,1513296000000000000,1,0,0,1,0,1,1,2,...,2,0,0,1,2,4,1,0,27,24,19,15,27001,0,1,1,0,0,0,0,0,0,0,6,1,0,1,0,1,0,0,1,0,0,0,0,1,31,acima da média,acima da média
27002020,5,9,4,10,6,3,36,3,1,3,0,3,1,0,46,5,9,1,2,11,2,16,21,5,22,16,2,5,10,7,11,1513296000000000000,2,0,0,1,0,0,0,3,...,0,0,0,0,0,8,0,0,9,15,9,9,27001,0,1,0,0,0,0,1,1,0,0,7,1,1,1,0,0,0,0,0,0,0,0,0,1,0,abaixo da média,abaixo da média
27002063,1,9,8,5,2,1,6,1,1,2,0,0,0,0,15,0,0,0,0,0,1,5,7,2,9,6,0,3,1,4,11,1513296000000000000,1,1,0,1,0,1,0,1,...,0,0,1,0,0,0,0,0,5,6,5,3,27001,0,1,0,0,0,0,0,1,0,1,7,0,1,0,0,0,0,0,0,0,1,0,0,0,0,acima da média,acima da média
27002950,4,14,9,14,13,6,26,5,0,3,8,1,0,0,55,8,4,0,1,7,1,25,17,2,37,20,1,10,7,16,11,1513296000000000000,2,0,0,1,0,0,1,1,...,1,0,0,0,1,7,0,0,23,19,8,14,27001,1,1,0,0,0,0,1,1,0,1,6,0,1,1,0,1,0,0,0,0,0,0,0,1,0,acima da média,acima da média
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27226131,4,30,12,14,20,3,26,12,0,10,2,1,0,0,69,3,4,2,2,8,3,4,26,4,33,14,5,11,15,15,8,1513296000000000000,1,0,0,1,0,0,0,2,...,0,0,0,0,0,1,0,0,22,14,11,11,27004,0,1,1,0,0,0,1,1,1,1,10,1,1,1,1,1,0,0,0,0,0,0,0,1,5,abaixo da média,abaixo da média
27226239,2,26,6,9,7,1,16,8,2,5,2,2,2,0,44,1,6,2,1,14,4,9,28,1,20,16,1,0,10,9,2,1514505600000000000,2,0,0,1,0,0,0,4,...,1,1,0,0,0,0,0,1,22,11,15,8,27012,0,1,0,0,1,0,1,1,1,1,15,1,1,1,0,1,0,0,0,0,0,0,0,1,0,abaixo da média,abaixo da média
27226468,4,13,8,8,12,2,12,5,3,6,3,0,0,0,45,1,12,1,9,8,0,10,26,5,26,21,12,3,14,12,7,1514505600000000000,2,0,0,1,0,0,1,2,...,0,0,0,0,1,4,1,0,18,16,8,11,27008,0,0,0,0,1,0,1,1,1,0,10,1,1,1,0,1,1,0,0,0,0,0,0,1,0,acima da média,acima da média
27226808,1,7,10,11,7,3,19,3,1,3,0,0,0,0,22,0,5,1,1,3,0,5,9,0,11,11,9,5,8,10,3,1513209600000000000,1,0,0,1,0,1,0,3,...,0,0,0,0,0,1,1,0,9,6,11,7,27005,0,1,0,0,0,0,1,1,1,0,10,0,1,1,1,1,0,0,0,0,0,0,0,1,6,acima da média,acima da média


In [ ]:
dados_para_exportar = dados_para_exportar.loc[dados_para_exportar['Valor Real Ideb'] ==  dados_para_exportar['Previsão Ideb']]


dados_para_exportar

,TX_RESP_Q030_B,TX_RESP_Q026_C,TX_RESP_Q035_C,TX_RESP_Q041_C,TX_RESP_Q042_B,TX_RESP_Q042_C,TX_RESP_Q046_A,TX_RESP_Q054_B,TX_RESP_Q051_C,TX_RESP_Q053_C,TX_RESP_Q059_C,TX_RESP_Q005_E,TX_RESP_Q006_D,TX_RESP_Q006_E,TX_RESP_Q007_A,TX_RESP_Q008_A,TX_RESP_Q008_C,TX_RESP_Q010_C,TX_RESP_Q013_C,TX_RESP_Q017_B,TX_RESP_Q017_C,TX_RESP_Q036_E,TX_RESP_Q037_E,TX_RESP_Q002_F,TX_RESP_Q016_F,TX_RESP_Q019_G,TX_RESP_Q004_B,TX_RESP_Q004_H,TX_RESP_Q003_A,TX_RESP_Q003_C,CO_ORGAO_REGIONAL,DT_ANO_LETIVO_TERMINO,TP_REGULAMENTACAO,IN_ESGOTO_REDE_PUBLICA,IN_BANHEIRO_FORA_PREDIO,IN_BANHEIRO_DENTRO_PREDIO,IN_EQUIP_VIDEOCASSETE,IN_EQUIP_PARABOLICA,IN_EQUIP_IMPRESSORA_MULT,NU_EQUIP_IMPRESSORA,...,TX_RESP_Q007_D,TX_RESP_Q008_D,TX_RESP_Q008_E,TX_RESP_Q011_E,TX_RESP_Q012_D,TX_RESP_Q014_A,TX_RESP_Q017_D,TX_RESP_Q017_E,TX_RESP_Q036_C,TX_RESP_Q003_E,TX_RESP_Q003_G,TX_RESP_Q003_H,CO_MICRORREGIAO,IN_PREDIO_COMPARTILHADO,IN_AGUA_FILTRADA,IN_AGUA_CACIMBA,IN_LIXO_QUEIMA,IN_SALA_ATENDIMENTO_ESPECIAL,IN_QUADRA_ESPORTES_DESCOBERTA,IN_BIBLIOTECA_SALA_LEITURA,IN_BANHEIRO_PNE,IN_AUDITORIO,IN_PATIO_DESCOBERTO,NU_SALAS_EXISTENTES,IN_EQUIP_COPIADORA,IN_EQUIP_SOM,IN_EQUIP_MULTIMIDIA,IN_EQUIP_FAX,IN_EQUIP_FOTO,TP_AEE,TP_ATIVIDADE_COMPLEMENTAR,IN_FINAL_SEMANA,IN_COMUM_FUND_AI,IN_COMUM_FUND_AF,IN_COMUM_MEDIO_NORMAL,IN_COMUM_EJA_FUND,IN_COMUM_EJA_MEDIO,QtdeDocentesFCEM,Valor Real Ideb,Previsão Ideb
27000095,1,9,9,20,7,4,17,3,0,1,0,0,0,1,27,9,1,0,1,7,0,15,14,2,33,16,1,5,9,6,11,1513296000000000000,1,1,0,1,0,0,0,2,...,0,0,1,0,1,4,1,1,12,9,9,11,27001,0,1,0,0,0,0,1,1,1,0,10,1,1,1,0,0,0,0,0,0,0,0,0,1,0,média,média
27001237,4,19,5,9,13,5,25,5,6,7,3,0,0,0,60,4,13,4,6,16,2,12,34,6,54,22,4,9,13,15,11,1513296000000000000,1,0,0,1,0,1,1,2,...,2,0,0,1,2,4,1,0,27,24,19,15,27001,0,1,1,0,0,0,0,0,0,0,6,1,0,1,0,1,0,0,1,0,0,0,0,1,31,acima da média,acima da média
27002020,5,9,4,10,6,3,36,3,1,3,0,3,1,0,46,5,9,1,2,11,2,16,21,5,22,16,2,5,10,7,11,1513296000000000000,2,0,0,1,0,0,0,3,...,0,0,0,0,0,8,0,0,9,15,9,9,27001,0,1,0,0,0,0,1,1,0,0,7,1,1,1,0,0,0,0,0,0,0,0,0,1,0,abaixo da média,abaixo da média
27002063,1,9,8,5,2,1,6,1,1,2,0,0,0,0,15,0,0,0,0,0,1,5,7,2,9,6,0,3,1,4,11,1513296000000000000,1,1,0,1,0,1,0,1,...,0,0,1,0,0,0,0,0,5,6,5,3,27001,0,1,0,0,0,0,0,1,0,1,7,0,1,0,0,0,0,0,0,0,1,0,0,0,0,acima da média,acima da média
27002950,4,14,9,14,13,6,26,5,0,3,8,1,0,0,55,8,4,0,1,7,1,25,17,2,37,20,1,10,7,16,11,1513296000000000000,2,0,0,1,0,0,1,1,...,1,0,0,0,1,7,0,0,23,19,8,14,27001,1,1,0,0,0,0,1,1,0,1,6,0,1,1,0,1,0,0,0,0,0,0,0,1,0,acima da média,acima da média
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27226131,4,30,12,14,20,3,26,12,0,10,2,1,0,0,69,3,4,2,2,8,3,4,26,4,33,14,5,11,15,15,8,1513296000000000000,1,0,0,1,0,0,0,2,...,0,0,0,0,0,1,0,0,22,14,11,11,27004,0,1,1,0,0,0,1,1,1,1,10,1,1,1,1,1,0,0,0,0,0,0,0,1,5,abaixo da média,abaixo da média
27226239,2,26,6,9,7,1,16,8,2,5,2,2,2,0,44,1,6,2,1,14,4,9,28,1,20,16,1,0,10,9,2,1514505600000000000,2,0,0,1,0,0,0,4,...,1,1,0,0,0,0,0,1,22,11,15,8,27012,0,1,0,0,1,0,1,1,1,1,15,1,1,1,0,1,0,0,0,0,0,0,0,1,0,abaixo da média,abaixo da média
27226468,4,13,8,8,12,2,12,5,3,6,3,0,0,0,45,1,12,1,9,8,0,10,26,5,26,21,12,3,14,12,7,1514505600000000000,2,0,0,1,0,0,1,2,...,0,0,0,0,1,4,1,0,18,16,8,11,27008,0,0,0,0,1,0,1,1,1,0,10,1,1,1,0,1,1,0,0,0,0,0,0,1,0,acima da média,acima da média
27226808,1,7,10,11,7,3,19,3,1,3,0,0,0,0,22,0,5,1,1,3,0,5,9,0,11,11,9,5,8,10,3,1513209600000000000,1,0,0,1,0,1,0,3,...,0,0,0,0,0,1,1,0,9,6,11,7,27005,0,1,0,0,0,0,1,1,1,0,10,0,1,1,1,1,0,0,0,0,0,0,0,1,6,acima da média,acima da média


In [ ]:
#dados_para_exportar.to_csv('C:/Users/maril/Documents/ufs/TCC e Pesquisa IDEB/resultado_analise_merge_dataset_completo.csv')
#dados_para_exportar.to_csv('/content/drive/MyDrive/TCC/resultado_analise_merge_dataset_completo.csv')
dados_para_exportar.to_csv('/content/drive/MyDrive/TCC/resultado_analise_merge_dataset_completo_AL.csv')

#### -Terceira análise: variáveis das escolas e professores

In [ ]:
columns_escolas = dataset.columns[237:404]

X_escolas = dataset[columns_escolas]

X_escolas

,TP_SITUACAO_FUNCIONAMENTO,DT_ANO_LETIVO_INICIO,DT_ANO_LETIVO_TERMINO,CO_REGIAO,CO_MESORREGIAO,CO_MICRORREGIAO,CO_UF,CO_MUNICIPIO,CO_DISTRITO,TP_DEPENDENCIA,TP_LOCALIZACAO,TP_REGULAMENTACAO,IN_LOCAL_FUNC_PREDIO_ESCOLAR,TP_OCUPACAO_PREDIO_ESCOLAR,IN_LOCAL_FUNC_SALAS_EMPRESA,IN_LOCAL_FUNC_SOCIOEDUCATIVO,IN_LOCAL_FUNC_UNID_PRISIONAL,IN_LOCAL_FUNC_PRISIONAL_SOCIO,IN_LOCAL_FUNC_TEMPLO_IGREJA,IN_LOCAL_FUNC_CASA_PROFESSOR,IN_LOCAL_FUNC_GALPAO,IN_LOCAL_FUNC_SALAS_OUTRA_ESC,IN_LOCAL_FUNC_OUTROS,IN_PREDIO_COMPARTILHADO,IN_AGUA_FILTRADA,IN_AGUA_REDE_PUBLICA,IN_AGUA_POCO_ARTESIANO,IN_AGUA_CACIMBA,IN_AGUA_FONTE_RIO,IN_AGUA_INEXISTENTE,IN_ENERGIA_REDE_PUBLICA,IN_ENERGIA_GERADOR,IN_ENERGIA_OUTROS,IN_ENERGIA_INEXISTENTE,IN_ESGOTO_REDE_PUBLICA,IN_ESGOTO_FOSSA,IN_ESGOTO_INEXISTENTE,IN_LIXO_COLETA_PERIODICA,IN_LIXO_QUEIMA,IN_LIXO_JOGA_OUTRA_AREA,...,IN_REGULAR,IN_EJA,IN_PROFISSIONALIZANTE,IN_COMUM_CRECHE,IN_COMUM_PRE,IN_COMUM_FUND_AI,IN_COMUM_FUND_AF,IN_COMUM_MEDIO_MEDIO,IN_COMUM_MEDIO_INTEGRADO,IN_COMUM_MEDIO_NORMAL,IN_ESP_EXCLUSIVA_CRECHE,IN_ESP_EXCLUSIVA_PRE,IN_ESP_EXCLUSIVA_FUND_AI,IN_ESP_EXCLUSIVA_FUND_AF,IN_ESP_EXCLUSIVA_MEDIO_MEDIO,IN_ESP_EXCLUSIVA_MEDIO_INTEGR,IN_ESP_EXCLUSIVA_MEDIO_NORMAL,IN_COMUM_EJA_FUND,IN_COMUM_EJA_MEDIO,IN_COMUM_EJA_PROF,IN_ESP_EXCLUSIVA_EJA_FUND,IN_ESP_EXCLUSIVA_EJA_MEDIO,IN_ESP_EXCLUSIVA_EJA_PROF,IN_COMUM_PROF,IN_ESP_EXCLUSIVA_PROF,QtdeDocentesEsp,QtdeDocentesMest,QtdeDocentesDout,QtdeDocentesSemPos,QtdeDocentesFCEM,QtdeDocentesDivSocCulta,QtdeDocentesAtendEsp,QtdeDocentesTotal,QtdeTurmasProf,QtdeDocentes,asQtdeDocentesEfetivos,QtdeDocentesTempo,QtdeDocentesComLicenciatura,QtdeAlunosTotal,Ideb-2017
27000095,1,1507161600000000000,1513296000000000000,2,2701,27001,27,2700102,270010205,2,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,...,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,62,0,0,103,0,0,0,172,172,172,24,148,172,508,3.4
27001237,1,1496361600000000000,1513296000000000000,2,2701,27001,27,2701605,270160505,2,1,1,1,1,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,...,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,111,0,0,89,31,0,0,250,250,250,11,225,250,792,3.9
27002020,1,1496448000000000000,1513296000000000000,2,2701,27001,27,2705002,270500205,2,1,2,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,...,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,100,0,0,86,0,0,0,222,222,222,5,217,222,652,3.3
27002063,1,1496448000000000000,1513296000000000000,2,2701,27001,27,2705002,270500205,2,2,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,...,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,20,0,0,43,0,0,0,73,73,73,13,40,63,309,3.8
27002950,1,1507075200000000000,1513296000000000000,2,2701,27001,27,2706422,270642205,2,1,2,1,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,...,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,72,7,0,36,0,0,0,121,121,121,37,84,117,442,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27226131,1,1496361600000000000,1513296000000000000,2,2701,27004,27,2700706,270070605,2,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,...,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,74,0,0,124,5,0,0,235,235,235,40,195,226,903,2.8
27226239,1,1486944000000000000,1514505600000000000,2,2703,27012,27,2702306,270230605,2,1,2,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,...,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,207,0,0,194,0,0,0,401,401,401,38,363,401,976,3.1
27226468,1,1486080000000000000,1514505600000000000,2,2703,27008,27,2708303,270830305,2,1,2,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,...,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,144,0,0,38,0,0,0,235,235,235,45,190,228,597,4.1
27226808,1,1501632000000000000,1513209600000000

In [ ]:
X_escolas.drop(columns=['DT_ANO_LETIVO_INICIO', 'DT_ANO_LETIVO_TERMINO'], axis=1, inplace=True)

X_transform = minmax(X_escolas)

X_train, X_test, y_train, y_test = train_test_split(X_transform, y, random_state=50, test_size=0.2)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 83.33 %
Naive Bayes: Acurácia do modelo: 50.0 %
Decision Tree: Acurácia do modelo: 100.0 %
SVM: Acurácia do modelo: 37.5 %
Logistic Regression: Acurácia do modelo: 45.83 %
KNN: Acurácia do modelo: 45.83 %


**Aplicando seleção de atributos (Filtragem)**

In [ ]:
X_selection = selecao_atributos(X_escolas, y, 100)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [  0   1   4  10  12  13  14  15  16  17  18  28  29  30  31  34  37  38
  39  40  53  54  68  72  87 111 118 119 121 122 125 132 133 134 135 136
 137 138 141 142 143 144 146 152 153] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 79.17 %
Naive Bayes: Acurácia do modelo: 50.0 %
Decision Tree: Acurácia do modelo: 100.0 %
SVM: Acurácia do modelo: 37.5 %
Logistic Regression: Acurácia do modelo: 45.83 %
KNN: Acurácia do modelo: 45.83 %


**Aplicando seleção de atributos (Embutida)**

In [ ]:
X_embedded = embedded_based(X, X_transform, y, 100)

X_embedded

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


,ID_UF,ID_MUNICIPIO,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,TX_RESP_Q020_A,TX_RESP_Q028_A,TX_RESP_Q028_B,TX_RESP_Q029_A,TX_RESP_Q029_B,TX_RESP_Q030_A,TX_RESP_Q030_B,TX_RESP_Q044_B,TX_RESP_Q018_B,TX_RESP_Q026_A,TX_RESP_Q026_B,TX_RESP_Q026_C,TX_RESP_Q032_B,TX_RESP_Q033_A,TX_RESP_Q033_C,TX_RESP_Q034_B,TX_RESP_Q034_C,TX_RESP_Q035_C,TX_RESP_Q040_A,TX_RESP_Q040_C,TX_RESP_Q041_B,TX_RESP_Q041_C,TX_RESP_Q042_A,TX_RESP_Q042_C,TX_RESP_Q046_A,TX_RESP_Q046_C,TX_RESP_Q047_C,TX_RESP_Q048_A,TX_RESP_Q048_C,TX_RESP_Q049_B,TX_RESP_Q049_C,TX_RESP_Q050_A,TX_RESP_Q050_B,TX_RESP_Q050_C,TX_RESP_Q054_B,TX_RESP_Q054_C,TX_RESP_Q055_A,TX_RESP_Q055_B,TX_RESP_Q055_C,TX_RESP_Q057_C,TX_RESP_Q058_C,TX_RESP_Q039_D,TX_RESP_Q045_A,TX_RESP_Q051_A,TX_RESP_Q051_B,TX_RESP_Q053_A,TX_RESP_Q053_B,TX_RESP_Q060_A,TX_RESP_Q060_B,TX_RESP_Q060_C,TX_RESP_Q005_E,TX_RESP_Q006_A,TX_RESP_Q006_D,TX_RESP_Q006_E,TX_RESP_Q007_C,TX_RESP_Q009_A,TX_RESP_Q009_B,TX_RESP_Q011_C,TX_RESP_Q012_D,TX_RESP_Q013_A,TX_RESP_Q013_C
27000095,27,2700102,2,1,75,87,8,82,13,93,1,16,9,38,48,9,55,9,30,46,6,9,94,1,49,20,85,4,17,3,3,84,1,35,2,63,20,1,3,2,60,29,2,4,2,2,70,88,6,71,22,11,45,39,0,31,0,1,3,64,32,2,1,80,1
27001237,27,2701605,2,1,144,167,16,162,21,179,4,36,30,103,60,19,98,25,63,91,18,5,178,2,52,9,163,5,25,19,17,121,12,62,16,133,17,14,5,5,130,33,4,8,6,1,104,137,28,98,70,27,64,76,0,66,0,0,5,115,65,4,2,145,6
27002020,27,2705002,2,1,78,88,9,89,8,91,5,8,20,62,26,9,60,16,25,46,4,4,87,7,34,10,84,3,36,3,5,73,3,36,3,72,10,2,3,0,52,24,0,1,2,3,73,85,2,58,24,20,40,26,3,33,1,0,11,61,37,1,0,78,2
27002063,27,2705002,2,2,22,32,4,33,3,35,1,7,2,13,13,9,15,3,21,18,3,8,36,0,15,5,33,1,6,3,2,29,3,12,3,21,11,1,1,1,20,8,2,1,1,2,23,29,3,26,6,7,20,7,0,12,0,0,1,27,9,0,0,30,0
27002950,27,2706422,2,1,109,123,13,115,22,133,4,46,22,76,48,14,82,27,48,66,14,9,138,2,29,14,121,6,26,4,4,122,2,45,4,97,21,4,5,2,102,23,2,4,5,7,80,103,29,72,56,11,52,65,1,56,0,0,4,101,40,1,1,112,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27226131,27,2700706,2,1,108,141,13,136,18,150,4,21,19,69,55,30,88,23,55,84,18,12,144,10,40,14,129,3,26,5,7,116,1,66,3,133,12,0,12,0,103,41,1,2,0,0,83,136,17,75,66,21,89,42,1,78,0,0,1,99,53,1,0,107,2
27226239,27,2702306,2,1,92,108,9,102,15,115,2,17,17,46,44,26,57,16,46,50,20,6,112,4,13,9,106,1,16,4,4,91,3,49,3,101,9,2,8,2,79,25,3,4,1,0,48,99,15,50,57,25,65,27,2,46,2,0,6,73,41,3,0,90,1
27226468,27,2708303,2,1,102,133,15,118,31,143,4,23,20,88,50,13,88,12,62,75,11,8,144,4,49,8,136,2,12,12,7,129,6,60,7,116,16,5,5,2,113,22,2,3,3,3,71,117,23,83,61,25,91,35,0,51,0,0,4,108,40,0,1,106,9
27226808,27,2707602,2,1,60,69,9,63,15,77,1,13,9,56,14,7,42,11,29,39,10,10,76,1,26,11,68,3,19,3,2,71,0,39,2,46,27,2,3,0,66,9,0,1,2,3,56,65,11,48,27,20,45,13,0,22,0,0,1,64,13,0,0,65,1


In [ ]:
X_transform_em = minmax(X_embedded)

X_train, X_test, y_train, y_test = train_test_split(X_transform_em, y, random_state=50, test_size=0.2)

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 50.0 %
Naive Bayes: Acurácia do modelo: 62.5 %
Decision Tree: Acurácia do modelo: 41.67 %
SVM: Acurácia do modelo: 58.33 %
Logistic Regression: Acurácia do modelo: 70.83 %
KNN: Acurácia do modelo: 41.67 %


**Aplicando seleção de atributos (Wrapper)**

In [ ]:
X_wrapper = wrapper_based(X, X_transform, y, 100)

X_wrapper

,ID_UF,ID_MUNICIPIO,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,TX_RESP_Q001_B,TX_RESP_Q020_A,TX_RESP_Q021_A,TX_RESP_Q028_A,TX_RESP_Q028_B,TX_RESP_Q029_A,TX_RESP_Q029_B,TX_RESP_Q030_A,TX_RESP_Q030_B,TX_RESP_Q031_A,TX_RESP_Q044_B,TX_RESP_Q052_A,TX_RESP_Q018_B,TX_RESP_Q026_A,TX_RESP_Q026_B,TX_RESP_Q026_C,TX_RESP_Q032_A,TX_RESP_Q032_B,TX_RESP_Q032_C,TX_RESP_Q033_A,TX_RESP_Q033_B,TX_RESP_Q033_C,TX_RESP_Q034_A,TX_RESP_Q034_B,TX_RESP_Q034_C,TX_RESP_Q035_C,TX_RESP_Q040_A,TX_RESP_Q040_C,TX_RESP_Q041_A,TX_RESP_Q041_B,TX_RESP_Q041_C,TX_RESP_Q042_A,TX_RESP_Q042_B,TX_RESP_Q042_C,TX_RESP_Q046_A,TX_RESP_Q046_B,...,TX_RESP_Q058_A,TX_RESP_Q058_B,TX_RESP_Q058_C,TX_RESP_Q039_A,TX_RESP_Q039_B,TX_RESP_Q039_D,TX_RESP_Q045_A,TX_RESP_Q045_B,TX_RESP_Q045_D,TX_RESP_Q051_A,TX_RESP_Q051_B,TX_RESP_Q051_C,TX_RESP_Q053_A,TX_RESP_Q053_B,TX_RESP_Q053_C,TX_RESP_Q060_A,TX_RESP_Q060_B,TX_RESP_Q060_C,TX_RESP_Q005_E,TX_RESP_Q006_A,TX_RESP_Q006_D,TX_RESP_Q006_E,TX_RESP_Q007_B,TX_RESP_Q007_C,TX_RESP_Q009_A,TX_RESP_Q009_B,TX_RESP_Q010_B,TX_RESP_Q010_D,TX_RESP_Q010_E,TX_RESP_Q011_A,TX_RESP_Q011_B,TX_RESP_Q011_C,TX_RESP_Q012_A,TX_RESP_Q012_B,TX_RESP_Q012_C,TX_RESP_Q012_D,TX_RESP_Q012_E,TX_RESP_Q013_A,TX_RESP_Q013_B,TX_RESP_Q013_C
27000095,27,2700102,2,1,59,75,68,87,8,82,13,93,1,73,16,73,9,38,48,9,10,55,22,9,43,30,36,46,6,9,94,1,25,49,20,85,7,4,17,61,...,32,48,2,15,46,2,70,24,1,88,6,0,71,22,1,11,45,39,0,31,0,1,68,3,64,32,9,0,0,78,18,2,85,11,0,1,0,80,15,1
27001237,27,2701605,2,1,108,144,130,167,16,162,21,179,4,133,36,102,30,103,60,19,25,98,41,25,75,63,64,91,18,5,178,2,118,52,9,163,13,5,25,111,...,112,40,6,40,88,1,104,61,9,137,28,6,98,70,7,27,64,76,0,66,0,0,121,5,115,65,11,0,0,114,68,4,143,36,3,2,1,145,34,6
27002020,27,2705002,2,1,60,78,70,88,9,89,8,91,5,73,8,61,20,62,26,9,18,60,13,16,48,25,38,46,4,4,87,7,48,34,10,84,6,3,36,43,...,55,18,2,28,35,3,73,16,2,85,2,1,58,24,3,20,40,26,3,33,1,0,44,11,61,37,15,0,0,66,34,1,72,28,0,0,0,78,20,2
27002063,27,2705002,2,2,24,22,15,32,4,33,3,35,1,26,7,29,2,13,13,9,4,15,14,3,8,21,12,18,3,8,36,0,15,15,5,33,2,1,6,23,...,10,21,1,0,18,2,23,10,1,29,3,1,26,6,2,7,20,7,0,12,0,0,19,1,27,9,2,0,0,24,11,0,25,11,0,0,0,30,6,0
27002950,27,2706422,2,1,72,109,88,123,13,115,22,133,4,98,46,83,22,76,48,14,15,82,31,27,48,48,45,66,14,9,138,2,93,29,14,121,13,6,26,91,...,79,35,5,21,95,7,80,55,4,103,29,0,72,56,3,11,52,65,1,56,0,0,83,4,101,40,9,0,0,101,41,1,120,20,1,1,1,112,29,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27226131,27,2700706,2,1,86,108,97,141,13,136,18,150,4,100,21,96,19,69,55,30,15,88,46,23,70,55,45,84,18,12,144,10,100,40,14,129,20,3,26,111,...,115,27,0,60,68,0,83,66,1,136,17,0,75,66,10,21,89,42,1,78,0,0,84,1,99,53,12,0,0,117,36,1,116,36,1,0,0,107,45,2
27226239,27,2702306,2,1,67,92,76,108,9,102,15,115,2,80,17,83,17,46,44,26,18,57,32,16,44,46,39,50,20,6,112,4,90,13,9,106,7,1,16,81,...,94,15,1,49,43,0,48,62,4,99,15,2,50,57,5,25,65,27,2,46,2,0,68,6,73,41,8,0,0,72,44,3,99,17,3,0,0,90,27,1
27226468,27,2708303,2,1,90,102,90,133,15,118,31,143,4,98,23,116,20,88,50,13,18,88,29,12,58,62,54,75,11,8,144,4,90,49,8,136,12,2,12,106,...,95,36,3,20,73,3,71,68,7,117,23,3,83,61,6,25,91,35,0,51,0,0,102,4,108,40,11,0,0,113,39,0,124,23,1,1,1,106,37,9
27226808,27,2707602,2,1,46,60,50,69,9,63,15,77,1,55,13,59,9,56,14,7,11,42,22,11,35,29,27,39,10,10,76,1,39,26,11,68,7,3,19,50,...,31,40,2,10,48,3,56,19,0,65,11,1,48,27,3,20,45,13,0,22,0,0,55,1,64,13,4,0,0,61,17,0,68,9,0,0,0,65,12,1


In [ ]:
X_transform_w = minmax(X_wrapper)

X_train, X_test, y_train, y_test = train_test_split(X_transform_w, y, random_state=50, test_size=0.2)

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 54.17 %
Naive Bayes: Acurácia do modelo: 58.33 %
Decision Tree: Acurácia do modelo: 50.0 %
SVM: Acurácia do modelo: 54.17 %
Logistic Regression: Acurácia do modelo: 54.17 %
KNN: Acurácia do modelo: 41.67 %
